<center>

# ~~~ The Matrix Elements ~~~

</center>


<center><img src="./images/qdef-banner.png" style="width: 500px">



<div style="max-width:600px; word-wrap:break-word;"

- Code the necessary operator algebra for determining matrix elements of one-electron and two-electron operators: <span style="color: #ff0000"> double_electron_braket, single_electron_braket </span>

- Code an algorithm to determine for a given number of electrons which terms will participate in their description:  <span style="color: #ff0000"> config_layout</span>

- Rehash the function for determining four-symbol braket identities: <span style="color: #ff0000"> braket_identities </span>

All of these functions were added to qdef.py on Dec-17 2021-12-17 14:58:45.
</div>

</center>




In [1]:
%load_ext line_profiler
%load_ext autoreload
# provide cell timings
%load_ext autotime
%autoreload 2
import sympy as sp
import numpy as np
%config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from misc import *
from IPython.display import display, Math, Latex
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
import time
from itertools import product
from itertools import permutations
import re
from sympy import Eijk as εijk

Reloading /Users/juan/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


# Examples

## config_layout

<center>

<div style="align:justtify; max-width:500px; word-wrap:break-word;"

For a given number of electrons in a given atomic orbital one would like to know how these might be rearranged within the irreducible representation of the crystal field potential.

This arrangement needs to take into account the crystal-field splitting of the given orbital angular momentum, the dimensions of the irreducible representations, and the spin of the electrons.

If a given irrep is included more than once then additional irrep symbols need to be created.

For example here's the code that shows the different crystal-configurations that correspond to the O group for different numbers of electrons.

</div>

</center>

In [2]:
rows = []
for num_electrons in range(1,11):
    row = [num_electrons]
    for config in config_layout('O',2,num_electrons):
        row.append(reduce(sp.Mul,[conf[0]**conf[1] for conf in config]))
    rows.append(row)
    display(sp.Matrix(row).T)


Matrix([[1, T_2, E]])

Matrix([[2, T_2**2, E*T_2, E**2]])

Matrix([[3, T_2**3, E*T_2**2, E**2*T_2, E**3]])

Matrix([[4, T_2**4, E*T_2**3, E**2*T_2**2, E**3*T_2, E**4]])

Matrix([[5, T_2**5, E*T_2**4, E**2*T_2**3, E**3*T_2**2, E**4*T_2]])

Matrix([[6, T_2**6, E*T_2**5, E**2*T_2**4, E**3*T_2**3, E**4*T_2**2]])

Matrix([[7, E*T_2**6, E**2*T_2**5, E**3*T_2**4, E**4*T_2**3]])

Matrix([[8, E**2*T_2**6, E**3*T_2**5, E**4*T_2**4]])

Matrix([[9, E**3*T_2**6, E**4*T_2**5]])

Matrix([[10, E**4*T_2**6]])

## braket_identities & double_electron braket

<div style="max-width:500px; word-wrap:break-word; text-align:justify; font-size:120%; margin:50px; border:2px dashed red; padding: 20px" <br>
For a given group, many four symbol brakets will be identical for an operator that has the symmetry of the group.
</div>

In [3]:
four_real_var_ids, four_symbol_ids, two_real_var_ids, two_symbol_ids = braket_identities('O')

O Creating all 4-symbol identities ...
O Refining set of identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 4-symbol brakets ...
O Creating a dictionary with all the 4-symbol replacements ...
O Creating all 2-symbol identities ...
O Creating set of 2 symbol identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 2-symbol brakets ...
O Creating a dictionary with all the 2-symbol replacements ...


In [4]:
def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({four_real_var_ids[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in four_symbol_ids:
            true_qet += v*Qet(four_symbol_ids[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

<div style="max-width:500px; word-wrap:break-word; text-align:justify; font-size:120%; margin:50px; border:2px dashed red; padding: 20px" <br>
As an example let's consider the electron configuration t_2^3, compute the brakets for the Coulomb repulsion between the enclosed states, and simplify them with the above identities.
</div>

In [5]:
t23 = CrystalElectronsSCoupling('O',[sp.Symbol('T_2')]*3)

In [6]:
terms = {}
for wave_key, wave_qet in t23.equiv_waves.items():
    aterm = wave_key.terms[-1]
    if aterm not in terms:
        terms[aterm] = {}
    terms[aterm][wave_key] = wave_qet
terms = OrderedDict(sorted(list(terms.items()), key=lambda x: x[0][0]))
for term, term_qets in terms.items():
    term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
    a_key = list(term_qets.keys())[0]
    a_qet = term_qets[a_key]
    matrix_element = double_electron_braket(a_qet,a_qet)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
    matrix_element = simplifier(matrix_element)
    display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<div style="max-width:500px; word-wrap:break-word; text-align:justify; font-size:120%; margin:50px; border:2px dashed red; padding: 20px" <br>
As a more complex example, let us consider all the crystal-electron configurations for the free ion electron configuration d^3.
</div>

In [7]:
def composite_symbol(x):
    return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
def fourtuplerecovery(ft):
    '''the inverse of composite_symbol'''
    return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))

In [8]:
# there are additional constraints that are not included in the
# braket identities, but which may be included by taking into account
# that all the brakets for qets in the same term need to be equal
# this allows a final simplification
# below, there's first a "dry run" where these additional identities
# are computed, and then a second one where this additional simplifications
# are enforced
group_label = 'O'
crystal_configs = config_layout(group_label, 2, 3)
all_term_vars = set()
for crystal_config in crystal_configs:
    Γ1s = crystal_config[0]
    e1s = sp.Symbol(sp.latex(Γ1s[0]).lower())**Γ1s[1]
    if len(crystal_config) == 1:
        Γ2s = []
        e_con = e1s
    else:
        Γ2s = crystal_config[1]
        e2s = sp.Symbol(sp.latex(Γ2s[0]).lower())**Γ2s[1]
        e_con = e1s * e2s
    display(Math(r'\Huge{%s}' % sp.latex(e_con)))
    wave_funs = CrystalElectronsLLcoupling(group_label, Γ1s, Γ2s)
    terms = {}
    for wave_key, wave_qet in wave_funs.equiv_waves.items():
        aterm = wave_key.terms[-1]
        if aterm not in terms:
            terms[aterm] = {}
        terms[aterm][wave_key] = wave_qet
    terms = OrderedDict(sorted(list(terms.items()), key=lambda x: x[0][0]))
    all_sols = {}
    # the dry run
    for term, term_qets in terms.items():
        term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
        the_diagonal = []
        for a_key, a_qet in term_qets.items():
            matrix_element = double_electron_braket(a_qet,a_qet)
            matrix_element = simplifier(matrix_element)
            the_diagonal.append(matrix_element)
        equations = []
        for qet0 in the_diagonal:
            for qet1 in the_diagonal:
                identity = qet0 - qet1
                if len(identity.dict) !=0:
                    equations.append(identity)
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in equations],[])))
        big_ma = [qet.vec_in_basis(problem_vars) for qet in equations] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        all_sols.update(sol_dict)
    def final_simplify(qet):
        new_qet = Qet({})
        for k, v in qet.dict.items():
            if k in all_sols.keys():
                new_qet = new_qet + v*Qet(all_sols[k])
            else:
                new_qet = new_qet + Qet({k:v})
        return new_qet
    print("="*10)
    term_vars = set()
    # the print run
    for term, term_qets in terms.items():
        term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
        for a_key, a_qet in term_qets.items():
            matrix_element = double_electron_braket(a_qet,a_qet)
            matrix_element = simplifier(matrix_element)
            term_vars.update(matrix_element.dict)
            matrix_element = final_simplify(matrix_element)
            display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))
            break
    print(len(term_vars),'independent integrals')
    all_term_vars.update(term_vars)
print("+"*20)
print(len(all_term_vars),'independent integrals overall')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

9 independent integrals


<IPython.core.display.Math object>

<IPython.core.display.Math object>

3 independent integrals
++++++++++++++++++++
12 independent integrals overall


In [9]:
# crystal_configs = config_layout('O',2,2)
# for crystal_config in crystal_configs:
#     Γ1s = crystal_config[0]
#     e1s = sp.Symbol(sp.latex(Γ1s[0]).lower())**Γ1s[1]
#     if len(crystal_config) == 1:
#         Γ2s = []
#         e_con = e1s
#     else:
#         Γ2s = crystal_config[1]
#         e2s = sp.Symbol(sp.latex(Γ2s[0]).lower())**Γ2s[1]
#         e_con = e1s * e2s
#     display(Math(r'\Huge{%s}' % sp.latex(e_con)))
#     wave_funs = CrystalElectronsLLcoupling('O', Γ1s, Γ2s)
#     terms = {}
#     for wave_key, wave_qet in wave_funs.equiv_waves.items():
#         aterm = wave_key.terms[-1]
#         if aterm not in terms:
#             terms[aterm] = {}
#         terms[aterm][wave_key] = wave_qet
#     terms = OrderedDict(sorted(list(terms.items()), key=lambda x: x[0][0]))
#     for term, term_qets in terms.items():
#         term_symbol = sp.Symbol('{}^{%d}%s' % (term[0]*2+1, sp.latex(term[1])))
#         a_key = list(term_qets.keys())[0]
#         a_qet = term_qets[a_key]
#         matrix_element = double_electron_braket(a_qet,a_qet)
#         matrix_element = simplifier(matrix_element)
#         display(Math(r'%s \rightarrow %s' % (sp.latex(term_symbol),sp.latex(matrix_element.as_braket()))))

<div style="max-width:500px; word-wrap:break-word; text-align:justify; font-size:120%; margin:50px; border:2px dashed red; padding: 20px" <br>
t2^2 looks ok, the other two look dicey.
</div>

In [10]:
# # the wave function looks certainly "plausible" (c.f. Table 2.3, and different component labels)
# for qet_key, qet_wave in wave_funs.equiv_waves.items():
#     print(qet_key)
#     display(qet_wave.as_ket())

In [94]:
# let's concentrate on finding the braket for the last wave function
qet_key, qet_wave = list(wave_funs.equiv_waves.items())[0]

In [95]:
matrix_element = double_electron_braket(qet_wave, qet_wave)
display(matrix_element.as_braket())
matrix_element = simplifier(matrix_element)
display(matrix_element.as_braket())

<{\gamma}{\gamma}|{\gamma}{\gamma}>/2 + <{\gamma}{\gamma}|{\zeta}{\zeta}>/2 + <{\zeta}{\zeta}|{\gamma}{\gamma}>/2 + <{\zeta}{\zeta}|{\zeta}{\zeta}>/2

<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>

## Docstrings 

In [4]:
double_electron_braket?

Signature: double_electron_braket(qet0, qet1, erase_spin=True)
Docstring:
Given  two  qets,  which  are  assumed to be composed of determinantal
states, and a two electron operator op, return value of the braket

  <qet0| \sum_{i>j=1}^N f_i,j |qet1> 

in terms of brakets of double electron orbitals.

Spin is assumed to be integrated in the notation for the symbols where
a  symbol  that  is  adorned with an upper bar is assumed to have spin
down and one without to have spin up.

This function assumes that the operator does not act on spin.

Parameters
----------
qet0    (qdefcore.Qet): a qet of determinantal states
qet1    (qdefcore.Qet): a qet of determinantal states
strip_spin      (bool): if True then spin bars are removed in output

Returns
-------
full_braket  (qdefcore.Qet):  with each key having five symbols, first
two equal to a two electron orbitals, middle one equal to the provided
double  electron  operator,  and the last two equal to another pair of
two  single  electron  or

In [5]:
single_electron_braket?

Signature: single_electron_braket(qet0, qet1, erase_spin=True)
Docstring:
Given  two qets, assumed to be composed of determinantal states, and a
single-electron operator return the value of the braket

  <qet0| \sum_1^N op_i |qet1>

in terms of brakets of single electron orbitals.

Spin is assumed to be integrated in the notation for the symbols where
a  symbol  that  is  adorned with an upper bar is assumed to have spin
down and one without to have spin up.

This function assumes that the operator does not act on spin.

Parameters
----------
qet0       (qdefcore.Qet): another qet
qet1       (qdefcore.Qet): a qet
erase_spin (bool)        : if True then spin bars are removed in output

Returns
-------
full_braket  (qdefcore.Qet): with each key having three symbols, first
one  equal  to  a  single  electron  orbital,  second one equal to the
provided  single electron operator, and the third one equal to another
single electron orbital. Interpreted as <φi | (op)* | φj>.

*  The  operator 

In [7]:
braket_identities?

Reloading /Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


Signature: braket_identities(group_label, verbose=True)
Docstring:
Given  a  group  many  two-electron  operator brakets yield equivalent
results for a spherically symmetric operator.

Returns
-------
+  real_var_full_simplifier : if the basis functions are real, one may
swap  symbols  from  the bra side over to the ket side and vice-versa.
super_solution_4 : the keys in this dictionary represent dependent

+  four-symbol brakets, and the corresponding values represent to what
they  equal in terms of the smallest possible set of independent four-
symbol brakets.

+  real_var_simplifiers_2  :  if the basis functions are real, one may
swap symbols from the bra side over to the ket side and vice-versa.

+  super_solution_2  : the keys in this dictionary represent dependent
two-symbol  brakets,  and  the  corresponding values represent to what
they  equal  in terms of the smallest possible set of independent two-
symbol brakets.
File:      /Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qde

In [8]:
config_layout?

Reloading /Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


Signature: config_layout(group_label, orbital_l, num_electrons)
Docstring:
Given  a  group label and a given number of electrons in the
given    orbital,    determine    which   crystal   electron
configurations are allowed.

Parameters
----------
group_label   (str)
orbita_l      (int)
num_electrons (int)

Returns
-------
configs  (list):  a list with two-tuples whose first element
is  a  label  for  an  irreducible  representation and whose
second  element  gives  how  many electrons would be in that
irrep.  If  the  irrep  may  appear  more than once then the
irreducible representation symbols are decorated by a number
of diamond symbols.

At a maximum each crystal-orbital may host as many electrons
as  twice  the  dimension  of  the corresponding irreducible
representation, this because of the electron's spin.

Example
-------
C_1  has  a  single  irrep,  which is singly degenerate, and
which  would  figure  3  times  for  an  l=1  splitting. Two
electrons   may  be  configured  in

# Workbench (look at your own peril)

In [164]:
# def config_layout(group_label, orbital_l, num_electrons):
#     '''
#     Given  a  group label and a given number of electrons in the
#     given    orbital,    determine    which   crystal   electron
#     configurations are allowed.

#     Parameters
#     ----------
#     group_label   (str)
#     orbita_l      (int)
#     num_electrons (int)

#     Returns
#     -------
#     configs  (list):  a list with two-tuples whose first element
#     is  a  label  for  an  irreducible  representation and whose
#     second  element  gives  how  many electrons would be in that
#     irrep.  If  the  irrep  may  appear  more than once then the
#     irreducible representation symbols are decorated by a number
#     of diamond symbols.

#     At a maximum each crystal-orbital may host as many electrons
#     as  twice  the  dimension  of  the corresponding irreducible
#     representation, this because of the electron's spin.

#     Example
#     -------
#     C_1  has  a  single  irrep,  which is singly degenerate, and
#     which  would  figure  3  times  for  an  l=1  splitting. Two
#     electrons   may  be  configured  in  six  different  crystal
#     configurations.

#     >>  config_layout('C_{1}', 1, 2)

#     >>  [[(A^{\diamond\diamond\diamond}, 2)],
#         [(A^{\diamond\diamond}, 1), (A^{\diamond\diamond\diamond}, 1)],
#         [(A^{\diamond\diamond}, 2)],
#         [(A^{\diamond}, 1), (A^{\diamond\diamond\diamond}, 1)],
#         [(A^{\diamond}, 1), (A^{\diamond\diamond}, 1)],
#         [(A^{\diamond}, 2)]]
#     '''
#     cf_splits = l_splitter(group_label,orbital_l).dict
#     group = CPGs.get_group_by_label(group_label)
#     irrep_replicas = []
#     cf_splits = l_splitter(group_label, orbital_l).dict
#     irrep_dims = {}
#     for k,v in cf_splits.items():
#         if v > 1:
#             for replica in range(v):
#                 replic = sp.Symbol(sp.latex(k)+'^{%s}'%(r'\diamond'*(replica+1)))
#                 irrep_dims[replic] = group.irrep_dims[k]
#                 irrep_replicas.append(replic)
#         else:
#             irrep_replicas.append(k)
#             irrep_dims[k] = group.irrep_dims[k]
#     splits = []
#     max_electrons = {irrep: 2*irrep_dims[irrep] for irrep in irrep_replicas}
#     iters = [list(range(max_electrons[irrep]+1)) for irrep in irrep_replicas]
#     for nums in product(*iters):
#         if sum(nums) == num_electrons:
#             splits.append(nums)
#     configs = []
#     for split in splits:
#         config = [(irrep,mult) for mult, irrep in zip(split, irrep_replicas) if mult !=0]
#         if len(config) > 0:
#             configs.append(config)
#     return configs

In [ ]:
rows = []
for num_electrons in range(1,11):
    row = [num_electrons]
    for config in config_layout('O',2,num_electrons):
        row.append(reduce(sp.Mul,[conf[0]**conf[1] for conf in config]))
    rows.append(row)
    display(sp.Matrix(row).T)


In [206]:
# def as_braket_with_operator(qet):
#     '''
#     Construct a symbol for a braket that has an intermediate
#     symbol interpreted as an operator
#     '''
#     tot = 0
#     assert len(list(qet.dict.keys())[0]) % 2 == 1
#     for k,v in qet.dict.items():
#         bra = ''.join(list(map(sp.latex, k[:len(k)//2])))
#         ket = ''.join(list(map(sp.latex, k[len(k)//2+1:])))
#         op = sp.latex(k[len(k)//2])
#         p = v*sp.Symbol(r'\langle{%s}|\hat{%s}|{%s}\rangle' % (bra, op, ket))
#         tot += p
#     return tot

# def strip_spin(qet):
#     '''
#     Removes bars from all symbols in the keys for the given qet.
#     '''
#     qet_dict = qet.dict
#     fun = lambda x: sp.Symbol(re.sub(r'\\bar{(.*)}',r'\1',sp.latex(x)))
#     new_qet_dict = {}
#     for k,v in qet_dict.items():
#         sk = tuple(map(fun,k))
#         if sk not in new_qet_dict:
#             new_qet_dict[sk] = 0
#         new_qet_dict[sk] += v
#     return Qet(new_qet_dict)

In [10]:
# def simplify_qet(qet):
#     new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
#     return Qet(new_dict)
# def braket_identities(group_label, verbose=True):
#     '''
#     Given a group many two-electron operator brakets yield equivalent
#     results for a spherically symmetric operator.
#     This function returns 

#     Returns
#     -------
#     real_var_full_simplifier : if the basis functions are real, one may swap symbols from
#                                the bra side over to the ket side and vice-versa.
#     super_solution_4 : the keys in this dictionary represent dependent four-symbol brakets,
#                        and the corresponding values represent to what they equal in terms
#                        of the smallest possible set of independent four-symbol brakets.
#     real_var_simplifiers_2 : if the basis functions are real, one may swap symbols from
#                              the bra side over to the ket side and vice-versa.
#     super_solution_2 : the keys in this dictionary represent dependent two-symbol brakets,
#                        and the corresponding values represent to what they equal in terms
#                        of the smallest possible set of independent two-symbol brakets.
    
#     '''
#     group = CPGs.get_group_by_label(group_label)
#     component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}

#     def overlinesqueegee(s):
#         '''
#         Going back from the overline shorthand for spin down,
#         acting on a single set of quantum numbers.
#         '''
#         if 'overline' in str(s):
#             spin = -sp.S(1)/2
#             comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
#         else:
#             spin = sp.S(1)/2
#             comp = s
#         return (comp, spin)
#     def spin_restoration(qet):
#         '''
#         Going back from the overline shorthand for spin down,
#         acting on a qet.
#         '''
#         new_dict = {}
#         for k,v in qet.dict.items():
#             k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
#             new_dict[k] = v
#         return Qet(new_dict)

#     def composite_symbol(x):
#         return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
#     def fourtuplerecovery(ft):
#         '''the inverse of composite_symbol'''
#         return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
#     if verbose:
#         msg = group_label + " Creating all 4-symbol identities ..."
#         print(msg)

#     # this   integral_identities  dictionary  will  have  as  keys
#     # 4-tuples  of  irreps  and  its  values  will  be lists whose
#     # elements  are  2-tuples whose first elements are 4-tuples of
#     # irrep  components  and  whose values are qets whose keys are
#     # 4-tuples  of  irrep components and whose values are numeric.
#     # These 4-tuples represent a braket with the Coulomb repulsion
#     # operator in between.

#     integral_identities = {}
#     ir_mats = group.irrep_matrices
#     for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
#         # To simplify calculations this part
#         # may only be done over quadruples in a standard
#         # order.
#         # Whatever is missed here is then brough back in
#         # by the reality relations.
#         altorder1 = (ir3, ir2, ir1, ir4)
#         altorder2 = (ir1, ir4, ir3, ir2)
#         altorder3 = (ir3, ir4, ir1, ir2)
#         if (altorder1 in integral_identities) or\
#            (altorder2 in integral_identities) or\
#            (altorder3 in integral_identities):
#             continue
#         integral_identity_sector = []
#         components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
#         comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
#         for R in group.generators:
#             R_id = {}
#             for γ1, γ2, γ3, γ4 in product(*components):
#                 for γ1p, γ2p, γ3p, γ4p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
#                               ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
#                               ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
#                         if val== 0:
#                             continue
#                         key = (γ1, γ2, γ3, γ4)
#                         if key not in R_id.keys():
#                             R_id[key] = []
#                         R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
#             R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
#             R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
#             integral_identity_sector.extend(R_id_total)
#         integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

#     # For solving the linear system it is convenient
#     # to have everything on one side of the equation.
#     if verbose:
#         msg = group_label + " Refining set of identities ..."
#         print(msg)
    
#     identities = {}
#     for ircombo in integral_identities:
#         these_ids = []
#         for v in integral_identities[ircombo]:
#             lhs, rhs = v
#             diff = Qet({lhs:1}) - rhs
#             if len(diff.dict) > 0:
#                 these_ids.append(diff)
#         identities[ircombo] = these_ids

#     # If an equation has only one key, then
#     # that immediately means that that braket is zero.
#     if verbose:
#         msg = group_label + " Finding trivial zeros ..."
#         print(msg)
    
#     # first determine which ones have to be zero
#     better_identities = {irc:[] for irc in identities}
#     all_zeros = {}
#     for ircombo in identities:
#         zeros = []
#         for identity in identities[ircombo]:
#             if len(identity.dict) == 1:
#                 zeros.append((list(identity.dict.keys())[0]))
#             else:
#                 better_identities[ircombo].append(identity)
#         all_zeros[ircombo] = zeros

#     # use them to simplify things.
#     if verbose:
#         msg = group_label + " Using them to simplify things ..."
#         print(msg)
    
#     great_identities = {irc:[] for irc in identities}
#     for ircombo in better_identities:
#         for identity in better_identities[ircombo]:
#             new_qet = Qet({})
#             for k,v in identity.dict.items():
#                 if k in all_zeros[ircombo]:
#                     continue
#                 else:
#                     new_qet+= Qet({k: v})
#             if len(new_qet.dict) == 0:
#                 continue
#             great_identities[ircombo].append(new_qet)

#     # Inside of a four-symbol braket one may do three exchanges
#     # that must result in the same value. That if the wave
#     # functions are assumed to be real-valued.
    
#     if verbose:
#         msg = group_label + " Creating reality identities ..."
#         print(msg)
    
#     real_var_simplifiers = {irc:[] for irc in great_identities}
#     kprimes = set()
#     # this has to run over all the quadruples of irs
#     for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
#         real_var_simplifier = {}
#         ircombo = (ir1, ir2, ir3, ir4)
#         components = [component_labels[ir] for ir in ircombo]
#         for γ1, γ2, γ3, γ4 in product(*components):
#             k = (γ1, γ2, γ3, γ4)
#             kalt1 = (γ3, γ2, γ1, γ4)
#             kalt2 = (γ1, γ4, γ3, γ2)
#             kalt3 = (γ3, γ4, γ1, γ2)
#             if kalt1 in kprimes:
#                 real_var_simplifier[k] = kalt1
#             elif kalt2 in kprimes:
#                 real_var_simplifier[k] = kalt2
#             elif kalt3 in kprimes:
#                 real_var_simplifier[k] = kalt3
#             else:
#                 real_var_simplifier[k] = k
#                 kprimes.add(k)
#         real_var_simplifiers[ircombo] = real_var_simplifier

#     real_var_full_simplifier = {}
#     for ircombo in real_var_simplifiers:
#         real_var_full_simplifier.update(real_var_simplifiers[ircombo])

#     # For each 4-tuple of irreps
#     # create a system of symbolic solutions
#     # and let sympy solve that.
#     # For each 4-tuple of irreps
#     # the end result is a dictionary
#     # whose keys represent the dependent
#     # brakets and whose values are the
#     # relation that those dependent values
#     # have with the independent brakets.
#     # As such, when these dictionaries are
#     # used on an expression, everything should
#     # then be given in terms of indepedent brakets.

#     if verbose:
#         msg = group_label + " Solving for independent 4-symbol brakets ..."
#         print(msg)
    
#     all_sols = {irc:[] for irc in great_identities}
#     for ircombo in great_identities:
#         zeros = all_zeros[ircombo]
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols[ircombo] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         zero_addendum = {k:{} for k in all_zeros[ircombo]}
#         sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         sol_dict.update(zero_addendum)
#         all_sols[ircombo] = sol_dict

#     # This final dictionary is unnecessary but
#     # simplifies calling the replacements onto
#     # a symbolic expression.
#     if verbose:
#         msg = group_label + " Creating a dictionary with all the 4-symbol replacements ..."
#         print(msg)
    
#     super_solution = {}
#     for ircombo in all_sols:
#         super_solution.update(all_sols[ircombo])

#     twotuplerecovery = fourtuplerecovery
#     if verbose:
#         msg = group_label + " Creating all 2-symbol identities ..."
#         print(msg)

#     # this   integral_identities  dictionary  will  have  as  keys
#     # 2-tuples  of  irreps  and  its  values  will  be lists whose
#     # elements  are  2-tuples whose first elements are 2-tuples of
#     # irrep  components  and  whose values are qets whose keys are
#     # 2-tuples  of  irrep components and whose values are numeric.
#     # These 2-tuples represent a braket with the an invariant operator
#     # operator in between.

#     integral_identities_2 = {}
#     ir_mats = group.irrep_matrices
#     for ir1, ir2 in product(*([group.irrep_labels]*2)):
#         # To simplify calculations this part
#         # can only be done over quadruples in a standard
#         # order.
#         # Whatever is missed here is then brough back in
#         # by the reality relations.
#         altorder1 = (ir2, ir1, ir1, ir4)
#         if (altorder1 in integral_identities_2):
#             continue
#         integral_identity_sector = []
#         components = [component_labels[ir] for ir in [ir1, ir2]]
#         comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
#         for R in group.generators:
#             R_id = {}
#             for γ1, γ2 in product(*components):
#                 for γ1p, γ2p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
#                         if val== 0:
#                             continue
#                         key = (γ1, γ2)
#                         if key not in R_id.keys():
#                             R_id[key] = []
#                         R_id[key].append( Qet({(γ1p,γ2p): val}) )
#             R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
#             R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
#             integral_identity_sector.extend(R_id_total)
#         integral_identities_2[(ir1,ir2)] = integral_identity_sector

#     # For solving the linear system it is convenient
#     # to have everything on one side of the equation.
#     if verbose:
#         msg = group_label + " Creating set of 2 symbol identities ..."
#         print(msg)
    
#     identities_2 = {}
    
#     for ircombo in integral_identities_2:
#         these_ids = []
#         for v in integral_identities_2[ircombo]:
#             lhs, rhs = v
#             diff = Qet({lhs:1}) - rhs
#             if len(diff.dict) > 0:
#                 these_ids.append(diff)
#         identities_2[ircombo] = these_ids

#     # If an equation has only one term, then
#     # that immediately means that that term is zero.
#     if verbose:
#         msg = group_label + " Finding trivial zeros ..."
#         print(msg)
    
#     # first determine which ones have to be zero
#     better_identities_2 = {irc:[] for irc in identities_2}
#     all_zeros_2 = {}
#     for ircombo in identities_2:
#         zeros = []
#         for identity in identities_2[ircombo]:
#             if len(identity.dict) == 1:
#                 zeros.append((list(identity.dict.keys())[0]))
#             else:
#                 better_identities_2[ircombo].append(identity)
#         all_zeros_2[ircombo] = zeros

#     # use them to simplify things.
#     if verbose:
#         msg = group_label + " Using them to simplify things ..."
#         print(msg)
    
#     great_identities_2 = {irc:[] for irc in identities_2}
#     for ircombo in better_identities_2:
#         for identity in better_identities_2[ircombo]:
#             new_qet = Qet({})
#             for k,v in identity.dict.items():
#                 if k in all_zeros_2[ircombo]:
#                     continue
#                 else:
#                     new_qet+= Qet({k: v})
#             if len(new_qet.dict) == 0:
#                 continue
#             great_identities_2[ircombo].append(new_qet)

#     # Inside of a two-symbol braket one may do three exchanges
#     # that must result in the same value. That if the wave
#     # functions are assumed to be real-valued.
#     if verbose:
#         msg = group_label + " Creating reality identities ..."
#         print(msg)
#     real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
#     # this has to run over all the quadruples of irs
#     kprimes = set()
#     for ir1, ir2 in product(*([group.irrep_labels]*2)):
#         real_var_simplifier = {}
#         ircombo = (ir1, ir2)
#         components = [component_labels[ir] for ir in ircombo]
#         for γ1, γ2 in product(*components):
#             k = (γ1, γ2)
#             kalt = (γ2, γ1)
#             # If for a given key I find that its
#             # switched version has already been seen
#             # Then that key has to be mapped to be
#             # mapped to the key already present.
#             if kalt in kprimes:
#                 real_var_simplifier[k] = kalt
#             else:
#                 real_var_simplifier[k] = k
#                 kprimes.add(k)
#         real_var_simplifiers_2[ircombo] = real_var_simplifier

#     # For each 2-tuple of irreps
#     # create a system of symbolic solutions
#     # and let sympy solve that.
#     # For each 2-tuple of irreps
#     # the end result is a dictionary
#     # whose keys represent the dependent
#     # brakets and whose values are the
#     # relation that those dependent values
#     # have with the independent brakets.
#     # As such, when these dictionaries are
#     # used on an expression, everything should
#     # then be given in terms of indepedent brakets.
#     if verbose:
#         msg = group_label + " Solving for independent 2-symbol brakets ..."
#         print(msg)
#     all_sols_2 = {irc:[] for irc in great_identities_2}
    
#     for ircombo in great_identities_2:
#         zeros = all_zeros_2[ircombo]
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols[ircombo] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
#         sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         sol_dict.update(zero_addendum)
#         all_sols_2[ircombo] = sol_dict

#     # This final dictionary is unnecessary but
#     # simplifies calling the replacements onto
#     # a symbolic expression.
#     if verbose:
#         msg = group_label + " Creating a dictionary with all the 2-symbol replacements ..."
#         print(msg)
    
#     super_solution_2 = {}
#     for ircombo in all_sols_2:
#         super_solution_2.update(all_sols_2[ircombo])

#     return real_var_full_simplifier, super_solution, real_var_simplifiers_2, super_solution_2


In [207]:
# def double_electron_braket(qet0, qet1, erase_spin=True):
#     '''
#     Given  two  qets,  which  are  assumed to be composed of determinantal
#     states, and a two electron operator op, return value of the braket

#       <qet0| \sum_{i>j=1}^N f_i,j |qet1> 
    
#     in terms of brakets of double electron orbitals.

#     Spin is assumed to be integrated in the notation for the symbols where
#     a  symbol  that  is  adorned with an upper bar is assumed to have spin
#     down and one without to have spin up.

#     This function assumes that the operator does not act on spin.

#     Parameters
#     ----------
#     qet0    (qdefcore.Qet): a qet of determinantal states
#     qet1    (qdefcore.Qet): a qet of determinantal states
#     strip_spin      (bool): if True then spin bars are removed in output

#     Returns
#     -------
#     full_braket  (qdefcore.Qet):  with each key having five symbols, first
#     two equal to a two electron orbitals, middle one equal to the provided
#     double  electron  operator,  and the last two equal to another pair of
#     two  single  electron  orbitals;  interpreted as <φi, φj | (op)* | φk,
#     φl>. 
    
#     *  The  operator  is omitted and is assumed to be in the middle of the
#     four symbols.

#     References
#     ----------
#     -   "Multiplets of Transition-Metal Ions in Crystals", Chapter 3
#         Sugano, Tanabe, and Kamimura
#     '''

#     full_braket = []
#     for det0, coeff0 in qet0.dict.items():
#         num_electrons = len(det0)
#         set0 = set(det0)
#         for det1, coeff1 in qet1.dict.items():
#             # before giving a value to the braket it is necessary to align the symbols 
#             # in the determinantal states and keep track of the reordering sign
#             set1 = set(det1)
#             common_symbs = list(set0.intersection(set1))
#             different_symbs0 = [x for x in det0 if x not in common_symbs]
#             different_symbs1 = [x for x in det1 if x not in common_symbs]
#             # there are no repeat symbols in any determinantal state
#             newdet0 = different_symbs0 + common_symbs
#             newdet1 = different_symbs1 + common_symbs
#             ordering0 = [det0.index(x) for x in newdet0]
#             ordering1 = [det1.index(x) for x in newdet1]
#             extra_sign = εijk(*ordering0) * εijk(*ordering1)
#             total_coeff = extra_sign * coeff0 * coeff1
#             odet0, odet1 = newdet0, newdet1
#             double_brakets = []
#             comparisons = list(map(lambda x: x[0]==x[1], zip(odet0, odet1)))
#             if all(comparisons):
#                 # CASE I
#                 # print(1)
#                 for i in range(num_electrons):
#                     spin_up_0_i = 'bar' not in str(odet0[i])
#                     for j in range(i+1,num_electrons):
#                         # print(i,j)
#                         spin_up_0_j = 'bar' not in str(odet0[j])
#                         double_brakets.append(((odet0[i], odet0[j],
#                                                 odet0[i], odet0[j]),
#                                                 total_coeff))
#                         if (spin_up_0_i == spin_up_0_j): 
#                             double_brakets.append(((odet0[i], odet0[j],
#                                                     odet0[j], odet0[i]),
#                                                     -total_coeff))
#                 # print(double_brakets)
#             elif (odet0[0] != odet1[0]) and all(comparisons[1:]):
#                 # CASE II
#                 # print(2)
#                 spin_up_0_0 = ('bar' not in str(odet0[0]))
#                 spin_up_1_0 = ('bar' not in str(odet1[0]))
#                 for j in range(1,num_electrons):
#                     spin_up_0_j = ('bar' not in str(odet0[j]))
#                     if (spin_up_0_0 == spin_up_1_0):
#                         double_brakets.append(((odet0[0], odet0[j],
#                                                 odet1[0], odet0[j]),
#                                                 total_coeff))
#                     if (spin_up_0_0 == spin_up_0_j) and (spin_up_0_j == spin_up_1_0): 
#                         double_brakets.append(((odet0[0], odet0[j],
#                                                 odet0[j], odet1[0]),
#                                                 -total_coeff))
#             elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]) and all(comparisons[2:]): 
#                 # CASE III
#                 # print(3)
#                 spin_up_0_0 = ('bar' not in str(odet0[0]))
#                 spin_up_0_1 = ('bar' not in str(odet0[1]))
#                 spin_up_1_0 = ('bar' not in str(odet1[0]))
#                 spin_up_1_1 = ('bar' not in str(odet1[1]))
#                 if (spin_up_0_0 == spin_up_1_0) and (spin_up_0_1 == spin_up_1_1):
#                     double_brakets.append(((odet0[0], odet0[1],
#                                             odet1[0], odet1[1]),
#                                             total_coeff))
#                 if (spin_up_0_0 == spin_up_1_1) and (spin_up_0_1 == spin_up_1_0):
#                     double_brakets.append(((odet0[0], odet0[1],
#                                             odet1[1], odet1[0]),
#                                             -total_coeff))
#             elif not any(comparisons[:3]):
#                 # CASE IV
#                 # print(4)
#                 double_brakets = []
#             else:
#                 raise Exception("Ooops, This case shouldn't occur")
#             # print("Yielded %d terms" % len(double_brakets))
#             full_braket.extend(double_brakets)
#     # full_braket = Qet(dict(full_braket))
#     # print("full_braket len = ",len(full_braket))
#     # for k in full_braket:
#     #     print(k)
#     # print(full_braket)
#     full_braket = sum([Qet({k:v}) for k,v in full_braket],Qet({}))
#     if erase_spin:  
#         return strip_spin(full_braket)
#     else:
#         return full_braket

# def single_electron_braket(qet0, qet1, erase_spin=True):
#     '''
#     Given  two qets, assumed to be composed of determinantal states, and a
#     single-electron operator return the value of the braket
    
#       <qet0| \sum_1^N op_i |qet1>

#     in terms of brakets of single electron orbitals.

#     Spin is assumed to be integrated in the notation for the symbols where
#     a  symbol  that  is  adorned with an upper bar is assumed to have spin
#     down and one without to have spin up.

#     This function assumes that the operator does not act on spin.

#     Parameters
#     ----------
#     qet0       (qdefcore.Qet): another qet
#     qet1       (qdefcore.Qet): a qet
#     erase_spin (bool)        : if True then spin bars are removed in output

#     Returns
#     -------
#     full_braket  (qdefcore.Qet): with each key having three symbols, first
#     one  equal  to  a  single  electron  orbital,  second one equal to the
#     provided  single electron operator, and the third one equal to another
#     single electron orbital. Interpreted as <φi | (op)* | φj>.

#     *  The  operator  is omitted and is assumed to be in the middle of the
#     two symbols.

#     References
#     ----------
#     -   "Multiplets of Transition-Metal Ions in Crystals", Chapter 3
#         Sugano, Tanabe, and Kamimura
#     '''
#     full_braket = []
#     for det0, coeff0 in qet0.dict.items():
#         num_electrons = len(det0)
#         for det1, coeff1 in qet1.dict.items():
#             # before  given  value to the braket it is necessary
#             # to  align  the symbols in the determinantal states
#             # and keep track of the reordering sign
#             set0 = set(det0)
#             set1 = set(det1)
#             # there should be no repeat symbols in any determinantal state
#             assert len(set0) == len(det0) and len(set1) == len(det1), "There's something funny here ..."
#             common_symbs = list(set0.intersection(set1))
#             different_symbs0 = [x for x in det0 if x not in common_symbs]
#             different_symbs1 = [x for x in det1 if x not in common_symbs]
#             newdet0 = different_symbs0 + common_symbs
#             newdet1 = different_symbs1 + common_symbs
#             ordering0 = [det0.index(x) for x in newdet0]
#             ordering1 = [det1.index(x) for x in newdet1]
#             extra_sign = εijk(*ordering0) * εijk(*ordering1)
#             total_coeff = coeff0 * coeff1 * extra_sign
#             odet0 = newdet0
#             odet1 = newdet1
#             comparisons = list(map(lambda x: x[0]==x[1], zip(odet0, odet1)))
#             if all(comparisons):
#                 # CASE I
#                 single_brakets = [((φ, φ), total_coeff) for φ in odet0]
#             elif (odet0[0] != odet1[0]) and all(comparisons[1:]):
#                 # CASE II
#                 spin_up_0_0 = 'bar' in str(det0[0])
#                 spin_up_1_0 = 'bar' in str(det1[0])
#                 if spin_up_0_0 == spin_up_1_0:
#                     single_brakets = [((odet0[0],
#                                         odet1[0]), total_coeff)]
#                 else:
#                     single_brakets = []
#             elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]):
#                 # CASE III
#                 single_brakets = []
#             else:
#                 raise Exception("Ooops, This case shouldn't occur")
#             full_braket.extend(single_brakets)
#     full_braket = Qet(dict(full_braket))
#     if erase_spin:
#         return strip_spin(full_braket)
#     else:
#         return full_braket

In [12]:
four_real_var_ids, four_symbol_ids, two_real_var_ids, two_symbol_ids = braket_identities('O')

O Creating all 4-symbol identities ...
O Refining set of identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 4-symbol brakets ...
O Creating a dictionary with all the 4-symbol replacements ...
O Creating all 2-symbol identities ...
O Creating set of 2 symbol identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 2-symbol brakets ...
O Creating a dictionary with all the 2-symbol replacements ...


In [209]:
def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({four_real_var_ids[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in four_symbol_ids:
            true_qet += v*Qet(four_symbol_ids[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

In [7]:
t23 = CrystalElectronsSCoupling('O',[sp.Symbol('T_2')]*3)
aterm = {k:v for k,v in t23.equiv_waves.items() if k.terms[-1] == (sp.S(3)/2,sp.Symbol('A_2'))}
qet_key, qet = list(aterm.items())[0]
aqet = (double_electron_braket(qet,qet, True))#.as_braket()
simplifier(aqet).as_braket()

3*<{\xi}{\phi}|{\xi}{\phi}> - 3*<{\xi}{\xi}|{\phi}{\phi}>

In [210]:
t23 = CrystalElectronsSCoupling('O',[sp.Symbol('T_2')]*3)
aterm = {k:v for k,v in t23.equiv_waves.items() if k.terms[-1] == (sp.S(1)/2,sp.Symbol('E'))}
for i in range(len(aterm)):
    qet_key, qet = list(aterm.items())[i]
    print(qet_key)
    aqet = (double_electron_braket(qet,qet,True))#.as_braket()
    display(simplifier(aqet).as_braket())

Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (1, T_1), (1/2, E)), γ={\gamma}, S=1/2, M=-1/2)


3*<{\xi}{\phi}|{\xi}{\phi}>

Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (1, T_1), (1/2, E)), γ={\zeta}, S=1/2, M=-1/2)


3*<{\xi}{\phi}|{\xi}{\phi}>

Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (1, T_1), (1/2, E)), γ={\gamma}, S=1/2, M=1/2)


3*<{\xi}{\phi}|{\xi}{\phi}>

Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (1, T_1), (1/2, E)), γ={\zeta}, S=1/2, M=1/2)


3*<{\xi}{\phi}|{\xi}{\phi}>

In [211]:
t23 = CrystalElectronsSCoupling('O',[sp.Symbol('T_2')]*3)
aterm = {k:v for k,v in t23.equiv_waves.items() if k.terms[-1] == (sp.S(1)/2,sp.Symbol('T_1'))}
for i in range(len(aterm)):
    print(i)
    qet_key, qet = list(aterm.items())[i]
    if i == 0:
        qet0 = qet
    print(qet_key)
    # display(qet.as_ket())
    aqet = (double_electron_braket(qet,qet,True))#.as_braket()
    display(aqet.as_braket())
    display(simplifier(aqet).as_braket())
    if i == 4:
        qet1 = qet
        break

0
Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (0, E), (1/2, T_1)), γ={\eta}, S=1/2, M=-1/2)


<{\chi}{\phi}|{\chi}{\phi}>/2 - <{\chi}{\phi}|{\phi}{\chi}>/2 + <{\chi}{\xi}|{\chi}{\xi}> - <{\chi}{\xi}|{\xi}{\chi}>/2 + <{\phi}{\chi}|{\phi}{\chi}>/2 + <{\phi}{\phi}|{\phi}{\phi}>/2 - <{\phi}{\phi}|{\xi}{\xi}>/2 - <{\xi}{\xi}|{\phi}{\phi}>/2 + <{\xi}{\xi}|{\xi}{\xi}>/2

2*<{\xi}{\phi}|{\xi}{\phi}> - 2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

1
Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (0, E), (1/2, T_1)), γ={\eta}, S=1/2, M=1/2)


-<{\phi}{\chi}|{\chi}{\phi}>/2 + <{\phi}{\chi}|{\phi}{\chi}> + <{\phi}{\phi}|{\phi}{\phi}>/2 - <{\phi}{\phi}|{\xi}{\xi}>/2 - <{\xi}{\chi}|{\chi}{\xi}>/2 + <{\xi}{\chi}|{\xi}{\chi}> - <{\xi}{\xi}|{\phi}{\phi}>/2 + <{\xi}{\xi}|{\xi}{\xi}>/2

...

<{\chi}{\chi}|{\chi}{\chi}>/2 - <{\chi}{\chi}|{\xi}{\xi}>/2 - <{\phi}{\chi}|{\chi}{\phi}>/2 + <{\phi}{\chi}|{\phi}{\chi}> + <{\phi}{\xi}|{\phi}{\xi}> - <{\phi}{\xi}|{\xi}{\phi}>/2 - <{\xi}{\xi}|{\chi}{\chi}>/2 + <{\xi}{\xi}|{\xi}{\xi}>/2

2*<{\xi}{\phi}|{\xi}{\phi}> - 2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

4
Ψ(electrons=(T_2, T_2, T_2), terms=((1/2, T_2), (1/2, T_2), (1, T_1), (1/2, T_1)), γ={\mu}, S=1/2, M=-1/2)


<{\chi}{\chi}|{\chi}{\chi}>/2 - <{\chi}{\chi}|{\phi}{\phi}>/2 + <{\chi}{\xi}|{\chi}{\xi}>/2 - <{\chi}{\xi}|{\xi}{\chi}>/2 - <{\phi}{\phi}|{\chi}{\chi}>/2 + <{\phi}{\phi}|{\phi}{\phi}>/2 + <{\phi}{\xi}|{\phi}{\xi}>/2 + <{\xi}{\chi}|{\xi}{\chi}>/2 - <{\xi}{\phi}|{\phi}{\xi}>/2 + <{\xi}{\phi}|{\xi}{\phi}>/2

2*<{\xi}{\phi}|{\xi}{\phi}> - 2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

## Braket Refinery

In [ ]:
# def simplify_qet(qet):
#     new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
#     return Qet(new_dict)
# def braket_identities(group_label, verbose=True):
#     '''
#     Given a group many two-electron operator brakets yield equivalent
#     results for a spherically symmetric operator.
#     This function returns 

#     Returns
#     -------
#     real_var_full_simplifier : if the basis functions are real, one may swap symbols from
#                                the bra side over to the ket side and vice-versa.
#     super_solution_4 : the keys in this dictionary represent dependent four-symbol brakets,
#                        and the corresponding values represent to what they equal in terms
#                        of the smallest possible set of independent four-symbol brakets.
#     real_var_simplifiers_2 : if the basis functions are real, one may swap symbols from
#                              the bra side over to the ket side and vice-versa.
#     super_solution_2 : the keys in this dictionary represent dependent two-symbol brakets,
#                        and the corresponding values represent to what they equal in terms
#                        of the smallest possible set of independent two-symbol brakets.
    
#     '''
#     group = CPGs.get_group_by_label(group_label)
#     component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}

#     def overlinesqueegee(s):
#         '''
#         Going back from the overline shorthand for spin down,
#         acting on a single set of quantum numbers.
#         '''
#         if 'overline' in str(s):
#             spin = -sp.S(1)/2
#             comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
#         else:
#             spin = sp.S(1)/2
#             comp = s
#         return (comp, spin)
#     def spin_restoration(qet):
#         '''
#         Going back from the overline shorthand for spin down,
#         acting on a qet.
#         '''
#         new_dict = {}
#         for k,v in qet.dict.items():
#             k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
#             new_dict[k] = v
#         return Qet(new_dict)

#     def composite_symbol(x):
#         return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
#     def fourtuplerecovery(ft):
#         '''the inverse of composite_symbol'''
#         return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
#     if verbose:
#         msg = group_label + " Creating all 4-symbol identities ..."
#         print(msg)

#     # this   integral_identities  dictionary  will  have  as  keys
#     # 4-tuples  of  irreps  and  its  values  will  be lists whose
#     # elements  are  2-tuples whose first elements are 4-tuples of
#     # irrep  components  and  whose values are qets whose keys are
#     # 4-tuples  of  irrep components and whose values are numeric.
#     # These 4-tuples represent a braket with the Coulomb repulsion
#     # operator in between.

#     integral_identities = {}
#     ir_mats = group.irrep_matrices
#     for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
#         # To simplify calculations this part
#         # may only be done over quadruples in a standard
#         # order.
#         # Whatever is missed here is then brough back in
#         # by the reality relations.
#         altorder1 = (ir3, ir2, ir1, ir4)
#         altorder2 = (ir1, ir4, ir3, ir2)
#         altorder3 = (ir3, ir4, ir1, ir2)
#         if (altorder1 in integral_identities) or\
#            (altorder2 in integral_identities) or\
#            (altorder3 in integral_identities):
#             continue
#         integral_identity_sector = []
#         components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
#         comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
#         for R in group.generators:
#             R_id = {}
#             for γ1, γ2, γ3, γ4 in product(*components):
#                 for γ1p, γ2p, γ3p, γ4p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
#                               ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
#                               ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
#                         if val== 0:
#                             continue
#                         key = (γ1, γ2, γ3, γ4)
#                         if key not in R_id.keys():
#                             R_id[key] = []
#                         R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
#             R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
#             R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
#             integral_identity_sector.extend(R_id_total)
#         integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

#     # For solving the linear system it is convenient
#     # to have everything on one side of the equation.
#     if verbose:
#         msg = group_label + " Refining set of identities ..."
#         print(msg)
    
#     identities = {}
#     for ircombo in integral_identities:
#         these_ids = []
#         for v in integral_identities[ircombo]:
#             lhs, rhs = v
#             diff = Qet({lhs:1}) - rhs
#             if len(diff.dict) > 0:
#                 these_ids.append(diff)
#         identities[ircombo] = these_ids

#     # If an equation has only one key, then
#     # that immediately means that that braket is zero.
#     if verbose:
#         msg = group_label + " Finding trivial zeros ..."
#         print(msg)
    
#     # first determine which ones have to be zero
#     better_identities = {irc:[] for irc in identities}
#     all_zeros = {}
#     for ircombo in identities:
#         zeros = []
#         for identity in identities[ircombo]:
#             if len(identity.dict) == 1:
#                 zeros.append((list(identity.dict.keys())[0]))
#             else:
#                 better_identities[ircombo].append(identity)
#         all_zeros[ircombo] = zeros

#     # use them to simplify things.
#     if verbose:
#         msg = group_label + " Using them to simplify things ..."
#         print(msg)
    
#     great_identities = {irc:[] for irc in identities}
#     for ircombo in better_identities:
#         for identity in better_identities[ircombo]:
#             new_qet = Qet({})
#             for k,v in identity.dict.items():
#                 if k in all_zeros[ircombo]:
#                     continue
#                 else:
#                     new_qet+= Qet({k: v})
#             if len(new_qet.dict) == 0:
#                 continue
#             great_identities[ircombo].append(new_qet)

#     # Inside of a four-symbol braket one may do three exchanges
#     # that must result in the same value. That if the wave
#     # functions are assumed to be real-valued.
    
#     if verbose:
#         msg = group_label + " Creating reality identities ..."
#         print(msg)
    
#     real_var_simplifiers = {irc:[] for irc in great_identities}
#     kprimes = set()
#     # this has to run over all the quadruples of irs
#     for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
#         real_var_simplifier = {}
#         ircombo = (ir1, ir2, ir3, ir4)
#         components = [component_labels[ir] for ir in ircombo]
#         for γ1, γ2, γ3, γ4 in product(*components):
#             k = (γ1, γ2, γ3, γ4)
#             kalt1 = (γ3, γ2, γ1, γ4)
#             kalt2 = (γ1, γ4, γ3, γ2)
#             kalt3 = (γ3, γ4, γ1, γ2)
#             if kalt1 in kprimes:
#                 real_var_simplifier[k] = kalt1
#             elif kalt2 in kprimes:
#                 real_var_simplifier[k] = kalt2
#             elif kalt3 in kprimes:
#                 real_var_simplifier[k] = kalt3
#             else:
#                 real_var_simplifier[k] = k
#                 kprimes.add(k)
#         real_var_simplifiers[ircombo] = real_var_simplifier

#     real_var_full_simplifier = {}
#     for ircombo in real_var_simplifiers:
#         real_var_full_simplifier.update(real_var_simplifiers[ircombo])

#     # For each 4-tuple of irreps
#     # create a system of symbolic solutions
#     # and let sympy solve that.
#     # For each 4-tuple of irreps
#     # the end result is a dictionary
#     # whose keys represent the dependent
#     # brakets and whose values are the
#     # relation that those dependent values
#     # have with the independent brakets.
#     # As such, when these dictionaries are
#     # used on an expression, everything should
#     # then be given in terms of indepedent brakets.

#     if verbose:
#         msg = group_label + " Solving for independent 4-symbol brakets ..."
#         print(msg)
    
#     all_sols = {irc:[] for irc in great_identities}
#     for ircombo in great_identities:
#         zeros = all_zeros[ircombo]
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols[ircombo] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         zero_addendum = {k:{} for k in all_zeros[ircombo]}
#         sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         sol_dict.update(zero_addendum)
#         all_sols[ircombo] = sol_dict

#     # This final dictionary is unnecessary but
#     # simplifies calling the replacements onto
#     # a symbolic expression.
#     if verbose:
#         msg = group_label + " Creating a dictionary with all the 4-symbol replacements ..."
#         print(msg)
    
#     super_solution = {}
#     for ircombo in all_sols:
#         super_solution.update(all_sols[ircombo])

#     # def simplify_config_matrix(ir1ir2, S, ir3):
#     #     config_matrix = config_matrices[ir1ir2][(S,ir3)]
#     #     num_rows = len(config_matrix)
#     #     simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
#     #     return simple_matrix
    
#     # if verbose:
#     #     msg = group_label + " Simplifying configuration matrices ..."
#     #     pbar.set_description(msg)
#     #     pbar.refresh()
#     #     pbar.n += 1
    
#     # simple_config_matrices = {k:{} for k in config_matrices}
#     # for ir1ir2 in config_matrices.keys():
#     #     for term_key in config_matrices[ir1ir2]:
#     #         S, Γ3 = term_key
#     #         simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

#     twotuplerecovery = fourtuplerecovery
#     if verbose:
#         msg = group_label + " Creating all 2-symbol identities ..."
#         print(msg)

#     # this   integral_identities  dictionary  will  have  as  keys
#     # 2-tuples  of  irreps  and  its  values  will  be lists whose
#     # elements  are  2-tuples whose first elements are 2-tuples of
#     # irrep  components  and  whose values are qets whose keys are
#     # 2-tuples  of  irrep components and whose values are numeric.
#     # These 2-tuples represent a braket with the an invariant operator
#     # operator in between.

#     integral_identities_2 = {}
#     ir_mats = group.irrep_matrices
#     for ir1, ir2 in product(*([group.irrep_labels]*2)):
#         # To simplify calculations this part
#         # can only be done over quadruples in a standard
#         # order.
#         # Whatever is missed here is then brough back in
#         # by the reality relations.
#         altorder1 = (ir2, ir1, ir1, ir4)
#         if (altorder1 in integral_identities_2):
#             continue
#         integral_identity_sector = []
#         components = [component_labels[ir] for ir in [ir1, ir2]]
#         comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
#         for R in group.generators:
#             R_id = {}
#             for γ1, γ2 in product(*components):
#                 for γ1p, γ2p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
#                         if val== 0:
#                             continue
#                         key = (γ1, γ2)
#                         if key not in R_id.keys():
#                             R_id[key] = []
#                         R_id[key].append( Qet({(γ1p,γ2p): val}) )
#             R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
#             R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
#             integral_identity_sector.extend(R_id_total)
#         integral_identities_2[(ir1,ir2)] = integral_identity_sector

#     # For solving the linear system it is convenient
#     # to have everything on one side of the equation.
#     if verbose:
#         msg = group_label + " Creating set of 2 symbol identities ..."
#         print(msg)
    
#     identities_2 = {}
    
#     for ircombo in integral_identities_2:
#         these_ids = []
#         for v in integral_identities_2[ircombo]:
#             lhs, rhs = v
#             diff = Qet({lhs:1}) - rhs
#             if len(diff.dict) > 0:
#                 these_ids.append(diff)
#         identities_2[ircombo] = these_ids

#     # If an equation has only one term, then
#     # that immediately means that that term is zero.
#     if verbose:
#         msg = group_label + " Finding trivial zeros ..."
#         print(msg)
    
#     # first determine which ones have to be zero
#     better_identities_2 = {irc:[] for irc in identities_2}
#     all_zeros_2 = {}
#     for ircombo in identities_2:
#         zeros = []
#         for identity in identities_2[ircombo]:
#             if len(identity.dict) == 1:
#                 zeros.append((list(identity.dict.keys())[0]))
#             else:
#                 better_identities_2[ircombo].append(identity)
#         all_zeros_2[ircombo] = zeros

#     # use them to simplify things.
#     if verbose:
#         msg = group_label + " Using them to simplify things ..."
#         print(msg)
    
#     great_identities_2 = {irc:[] for irc in identities_2}
#     for ircombo in better_identities_2:
#         for identity in better_identities_2[ircombo]:
#             new_qet = Qet({})
#             for k,v in identity.dict.items():
#                 if k in all_zeros_2[ircombo]:
#                     continue
#                 else:
#                     new_qet+= Qet({k: v})
#             if len(new_qet.dict) == 0:
#                 continue
#             great_identities_2[ircombo].append(new_qet)

#     # Inside of a two-symbol braket one may do three exchanges
#     # that must result in the same value. That if the wave
#     # functions are assumed to be real-valued.
#     if verbose:
#         msg = group_label + " Creating reality identities ..."
#         print(msg)
#     real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
#     # this has to run over all the quadruples of irs
#     kprimes = set()
#     for ir1, ir2 in product(*([group.irrep_labels]*2)):
#         real_var_simplifier = {}
#         ircombo = (ir1, ir2)
#         components = [component_labels[ir] for ir in ircombo]
#         for γ1, γ2 in product(*components):
#             k = (γ1, γ2)
#             kalt = (γ2, γ1)
#             # If for a given key I find that its
#             # switched version has already been seen
#             # Then that key has to be mapped to be
#             # mapped to the key already present.
#             if kalt in kprimes:
#                 real_var_simplifier[k] = kalt
#             else:
#                 real_var_simplifier[k] = k
#                 kprimes.add(k)
#         real_var_simplifiers_2[ircombo] = real_var_simplifier

#     # For each 2-tuple of irreps
#     # create a system of symbolic solutions
#     # and let sympy solve that.
#     # For each 2-tuple of irreps
#     # the end result is a dictionary
#     # whose keys represent the dependent
#     # brakets and whose values are the
#     # relation that those dependent values
#     # have with the independent brakets.
#     # As such, when these dictionaries are
#     # used on an expression, everything should
#     # then be given in terms of indepedent brakets.
#     if verbose:
#         msg = group_label + " Solving for independent 2-symbol brakets ..."
#         print(msg)
#     all_sols_2 = {irc:[] for irc in great_identities_2}
    
#     for ircombo in great_identities_2:
#         zeros = all_zeros_2[ircombo]
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols[ircombo] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
#         sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         sol_dict.update(zero_addendum)
#         all_sols_2[ircombo] = sol_dict

#     # This final dictionary is unnecessary but
#     # simplifies calling the replacements onto
#     # a symbolic expression.
#     if verbose:
#         msg = group_label + " Creating a dictionary with all the 2-symbol replacements ..."
#         print(msg)
    
#     super_solution_2 = {}
#     for ircombo in all_sols_2:
#         super_solution_2.update(all_sols_2[ircombo])

#     return real_var_full_simplifier, super_solution, real_var_simplifiers_2, super_solution_2

#     # for a given electron config
#     more_ids = {e_config:[] for e_config in simple_config_matrices}
#     for e_config in simple_config_matrices:
#         for term in simple_config_matrices[e_config]:
#             the_matrix = simple_config_matrices[e_config][term]
#             num_rows = len(the_matrix)
#             num_cols = len(the_matrix)
#             the_state_keys = config_supplement[e_config][term]
#             the_key_matrix = {}
#             for row_idx in range(num_rows):
#                 Γ3_row = the_state_keys[row_idx][2]
#                 γ3_row = the_state_keys[row_idx][3]
#                 for col_idx in range(num_cols):
#                     Γ3_col = the_state_keys[col_idx][2]
#                     γ3_col = the_state_keys[col_idx][3]
#                     matrix_val = the_matrix[row_idx][col_idx]
#                     the_key_matrix[(γ3_row,γ3_col)] = matrix_val
#             # now go over the keys of super_solution_2
#             # and if one of those keys matches with a key in the_key_matrix
#             # do something about it
#             for k in super_solution_2:
#                 if k in the_key_matrix:
#                     v = super_solution_2[k]
#                     # this matrix element
#                     matrix_element = the_key_matrix[k]
#                     # must be identified with the sum
#                     # as given in v
#                     matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
#                     identity = (matrix_element - matrix_equiv) #=0
#                     if len(identity.dict) != 0:
#                         more_ids[e_config].append(identity)
#                         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
#     if verbose:
#         msg = group_label + " Simplifying numeric values of qets ..."
#         pbar.set_description(msg)
#         pbar.refresh()
#         pbar.n += 1
    
#     more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}
    
#     if verbose:
#         msg = group_label + " Solving for dependent vars in terms of independent ones ..."
#         pbar.set_description(msg)
#         pbar.refresh()
#         pbar.n += 1
    
#     all_sols_2_4 = {irc:[] for irc in more_ids}
    
#     for e_config in more_ids:
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols_2_4[e_config] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         all_sols_2_4[e_config] = sol_dict

#     # flatten into a single dictionary of replacements
#     fab_solution_2_4 = {}
#     for e_config in all_sols_2_4:
#         fab_solution_2_4.update(all_sols_2_4[e_config])
#     if verbose:
#         msg = group_label + " There are %d less independent variables ..."
#         pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
#         pbar.refresh()
#         pbar.n += 1
    
#     def simplifier_f(qet):
#         true_qet = Qet({})
#         for k,v in qet.dict.items():
#             if k in fab_solution_2_4:
#                 true_qet += v*Qet(fab_solution_2_4[k])
#             else:
#                 true_qet += Qet({k:v})
#         return true_qet
    
#     def simplify_config_matrix_f(ir1ir2, S, ir3):
#         config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
#         num_rows = len(config_matrix)
#         simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
#         #simple_matrix = sp.Matrix(simple_matrix)
#         return simple_matrix
    
#     if verbose:
#         msg = group_label + " Making final simplifications ..."
#         pbar.set_description(msg)
#         pbar.refresh()
#         pbar.n += 1
    
#     final_config_matrices = {k:{} for k in config_matrices}
#     for ir1ir2 in config_matrices.keys():
#         for term_key in config_matrices[ir1ir2]:
#             S, Γ3 = term_key
#             final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)
    
#     if verbose:
#         msg = group_label + " Finished"
#         pbar.set_description(msg)
#         pbar.refresh()
    
#     return final_config_matrices

I need to rehash the code that I was using to generate 4-symbol identities.

In [36]:
# def simplify_qet(qet):
#     new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
#     return Qet(new_dict)
# def braket_identities(group_label, verbose=True):
#     '''
#     Given a group many two-electron operator brakets yield equivalent
#     results for a spherically symmetric operator.
#     This function returns 

#     Returns
#     -------
#     real_var_full_simplifier : if the basis functions are real, one may swap symbols from
#                                the bra side over to the ket side and vice-versa.
#     super_solution_4 : the keys in this dictionary represent dependent four-symbol brakets,
#                        and the corresponding values represent to what they equal in terms
#                        of the smallest possible set of independent four-symbol brakets.
#     real_var_simplifiers_2 : if the basis functions are real, one may swap symbols from
#                              the bra side over to the ket side and vice-versa.
#     super_solution_2 : the keys in this dictionary represent dependent two-symbol brakets,
#                        and the corresponding values represent to what they equal in terms
#                        of the smallest possible set of independent two-symbol brakets.
    
#     '''
#     group = CPGs.get_group_by_label(group_label)
#     component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}

#     def overlinesqueegee(s):
#         '''
#         Going back from the overline shorthand for spin down,
#         acting on a single set of quantum numbers.
#         '''
#         if 'overline' in str(s):
#             spin = -sp.S(1)/2
#             comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
#         else:
#             spin = sp.S(1)/2
#             comp = s
#         return (comp, spin)
#     def spin_restoration(qet):
#         '''
#         Going back from the overline shorthand for spin down,
#         acting on a qet.
#         '''
#         new_dict = {}
#         for k,v in qet.dict.items():
#             k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
#             new_dict[k] = v
#         return Qet(new_dict)

#     def composite_symbol(x):
#         return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
#     def fourtuplerecovery(ft):
#         '''the inverse of composite_symbol'''
#         return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
#     if verbose:
#         msg = group_label + " Creating all 4-symbol identities ..."
#         print(msg)

#     # this   integral_identities  dictionary  will  have  as  keys
#     # 4-tuples  of  irreps  and  its  values  will  be lists whose
#     # elements  are  2-tuples whose first elements are 4-tuples of
#     # irrep  components  and  whose values are qets whose keys are
#     # 4-tuples  of  irrep components and whose values are numeric.
#     # These 4-tuples represent a braket with the Coulomb repulsion
#     # operator in between.

#     integral_identities = {}
#     ir_mats = group.irrep_matrices
#     for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
#         # To simplify calculations this part
#         # may only be done over quadruples in a standard
#         # order.
#         # Whatever is missed here is then brough back in
#         # by the reality relations.
#         altorder1 = (ir3, ir2, ir1, ir4)
#         altorder2 = (ir1, ir4, ir3, ir2)
#         altorder3 = (ir3, ir4, ir1, ir2)
#         if (altorder1 in integral_identities) or\
#            (altorder2 in integral_identities) or\
#            (altorder3 in integral_identities):
#             continue
#         integral_identity_sector = []
#         components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
#         comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
#         for R in group.generators:
#             R_id = {}
#             for γ1, γ2, γ3, γ4 in product(*components):
#                 for γ1p, γ2p, γ3p, γ4p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
#                               ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
#                               ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
#                         if val== 0:
#                             continue
#                         key = (γ1, γ2, γ3, γ4)
#                         if key not in R_id.keys():
#                             R_id[key] = []
#                         R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
#             R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
#             R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
#             integral_identity_sector.extend(R_id_total)
#         integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

#     # For solving the linear system it is convenient
#     # to have everything on one side of the equation.
#     if verbose:
#         msg = group_label + " Creating set of identities ..."
#         print(msg)
    
#     identities = {}
#     for ircombo in integral_identities:
#         these_ids = []
#         for v in integral_identities[ircombo]:
#             lhs, rhs = v
#             diff = Qet({lhs:1}) - rhs
#             if len(diff.dict) > 0:
#                 these_ids.append(diff)
#         identities[ircombo] = these_ids

#     # If an equation has only one key, then
#     # that immediately means that that braket is zero.
#     if verbose:
#         msg = group_label + " Finding trivial zeros ..."
#         print(msg)
    
#     # first determine which ones have to be zero
#     better_identities = {irc:[] for irc in identities}
#     all_zeros = {}
#     for ircombo in identities:
#         zeros = []
#         for identity in identities[ircombo]:
#             if len(identity.dict) == 1:
#                 zeros.append((list(identity.dict.keys())[0]))
#             else:
#                 better_identities[ircombo].append(identity)
#         all_zeros[ircombo] = zeros

#     # use them to simplify things.
#     if verbose:
#         msg = group_label + " Using them to simplify things ..."
#         print(msg)
    
#     great_identities = {irc:[] for irc in identities}
#     for ircombo in better_identities:
#         for identity in better_identities[ircombo]:
#             new_qet = Qet({})
#             for k,v in identity.dict.items():
#                 if k in all_zeros[ircombo]:
#                     continue
#                 else:
#                     new_qet+= Qet({k: v})
#             if len(new_qet.dict) == 0:
#                 continue
#             great_identities[ircombo].append(new_qet)

#     # Inside of a four-symbol braket one may do three exchanges
#     # that must result in the same value. That if the wave
#     # functions are assumed to be real-valued.
    
#     if verbose:
#         msg = group_label + " Creating reality identities ..."
#         print(msg)
    
#     real_var_simplifiers = {irc:[] for irc in great_identities}
#     kprimes = set()
#     # this has to run over all the quadruples of irs
#     for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
#         real_var_simplifier = {}
#         ircombo = (ir1, ir2, ir3, ir4)
#         components = [component_labels[ir] for ir in ircombo]
#         for γ1, γ2, γ3, γ4 in product(*components):
#             k = (γ1, γ2, γ3, γ4)
#             kalt1 = (γ3, γ2, γ1, γ4)
#             kalt2 = (γ1, γ4, γ3, γ2)
#             kalt3 = (γ3, γ4, γ1, γ2)
#             if kalt1 in kprimes:
#                 real_var_simplifier[k] = kalt1
#             elif kalt2 in kprimes:
#                 real_var_simplifier[k] = kalt2
#             elif kalt3 in kprimes:
#                 real_var_simplifier[k] = kalt3
#             else:
#                 real_var_simplifier[k] = k
#                 kprimes.add(k)
#         real_var_simplifiers[ircombo] = real_var_simplifier

#     real_var_full_simplifier = {}
#     for ircombo in real_var_simplifiers:
#         real_var_full_simplifier.update(real_var_simplifiers[ircombo])

#     # For each 4-tuple of irreps
#     # create a system of symbolic solutions
#     # and let sympy solve that.
#     # For each 4-tuple of irreps
#     # the end result is a dictionary
#     # whose keys represent the dependent
#     # brakets and whose values are the
#     # relation that those dependent values
#     # have with the independent brakets.
#     # As such, when these dictionaries are
#     # used on an expression, everything should
#     # then be given in terms of indepedent brakets.

#     if verbose:
#         msg = group_label + " Solving for independent 4-symbol brakets ..."
#         print(msg)
    
#     all_sols = {irc:[] for irc in great_identities}
#     for ircombo in great_identities:
#         zeros = all_zeros[ircombo]
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols[ircombo] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         zero_addendum = {k:{} for k in all_zeros[ircombo]}
#         sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         sol_dict.update(zero_addendum)
#         all_sols[ircombo] = sol_dict

#     # This final dictionary is unnecessary but
#     # simplifies calling the replacements onto
#     # a symbolic expression.
#     if verbose:
#         msg = group_label + " Creating a dictionary with all the 4-symbol replacements ..."
#         print(msg)
    
#     super_solution = {}
#     for ircombo in all_sols:
#         super_solution.update(all_sols[ircombo])
#     # return super_solution

#     # def simplifier(qet):
#     #     simp_ket = Qet({})
#     #     for k,v in qet.dict.items():
#     #         simp_ket += Qet({real_var_full_simplifier[k]:v})
#     #     true_qet = Qet({})
#     #     for k,v in simp_ket.dict.items():
#     #         if k in super_solution:
#     #             true_qet += v*Qet(super_solution[k])
#     #         else:
#     #             true_qet += Qet({k:v})
#     #     return true_qet

#     # def simplify_config_matrix(ir1ir2, S, ir3):
#     #     config_matrix = config_matrices[ir1ir2][(S,ir3)]
#     #     num_rows = len(config_matrix)
#     #     simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
#     #     return simple_matrix
    
#     # if verbose:
#     #     msg = group_label + " Simplifying configuration matrices ..."
#     #     pbar.set_description(msg)
#     #     pbar.refresh()
#     #     pbar.n += 1
    
#     # simple_config_matrices = {k:{} for k in config_matrices}
#     # for ir1ir2 in config_matrices.keys():
#     #     for term_key in config_matrices[ir1ir2]:
#     #         S, Γ3 = term_key
#     #         simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

#     twotuplerecovery = fourtuplerecovery
#     if verbose:
#         msg = group_label + " Creating all 2-symbol identities ..."
#         print(msg)

#     # this   integral_identities  dictionary  will  have  as  keys
#     # 2-tuples  of  irreps  and  its  values  will  be lists whose
#     # elements  are  2-tuples whose first elements are 2-tuples of
#     # irrep  components  and  whose values are qets whose keys are
#     # 2-tuples  of  irrep components and whose values are numeric.
#     # These 2-tuples represent a braket with the an invariant operator
#     # operator in between.

#     integral_identities_2 = {}
#     ir_mats = group.irrep_matrices
#     for ir1, ir2 in product(*([group.irrep_labels]*2)):
#         # To simplify calculations this part
#         # can only be done over quadruples in a standard
#         # order.
#         # Whatever is missed here is then brough back in
#         # by the reality relations.
#         altorder1 = (ir2, ir1, ir1, ir4)
#         if (altorder1 in integral_identities_2):
#             continue
#         integral_identity_sector = []
#         components = [component_labels[ir] for ir in [ir1, ir2]]
#         comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
#         for R in group.generators:
#             R_id = {}
#             for γ1, γ2 in product(*components):
#                 for γ1p, γ2p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
#                         if val== 0:
#                             continue
#                         key = (γ1, γ2)
#                         if key not in R_id.keys():
#                             R_id[key] = []
#                         R_id[key].append( Qet({(γ1p,γ2p): val}) )
#             R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
#             R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
#             integral_identity_sector.extend(R_id_total)
#         integral_identities_2[(ir1,ir2)] = integral_identity_sector

#     # For solving the linear system it is convenient
#     # to have everything on one side of the equation.
#     if verbose:
#         msg = group_label + " Creating set of 2 symbol identities ..."
#         print(msg)
    
#     identities_2 = {}
    
#     for ircombo in integral_identities_2:
#         these_ids = []
#         for v in integral_identities_2[ircombo]:
#             lhs, rhs = v
#             diff = Qet({lhs:1}) - rhs
#             if len(diff.dict) > 0:
#                 these_ids.append(diff)
#         identities_2[ircombo] = these_ids

#     # If an equation has only one term, then
#     # that immediately means that that term is zero.
#     if verbose:
#         msg = group_label + " Finding trivial zeros ..."
#         print(msg)
    
#     # first determine which ones have to be zero
#     better_identities_2 = {irc:[] for irc in identities_2}
#     all_zeros_2 = {}
#     for ircombo in identities_2:
#         zeros = []
#         for identity in identities_2[ircombo]:
#             if len(identity.dict) == 1:
#                 zeros.append((list(identity.dict.keys())[0]))
#             else:
#                 better_identities_2[ircombo].append(identity)
#         all_zeros_2[ircombo] = zeros

#     # use them to simplify things.
#     if verbose:
#         msg = group_label + " Using them to simplify things ..."
#         print(msg)
    
#     great_identities_2 = {irc:[] for irc in identities_2}
#     for ircombo in better_identities_2:
#         for identity in better_identities_2[ircombo]:
#             new_qet = Qet({})
#             for k,v in identity.dict.items():
#                 if k in all_zeros_2[ircombo]:
#                     continue
#                 else:
#                     new_qet+= Qet({k: v})
#             if len(new_qet.dict) == 0:
#                 continue
#             great_identities_2[ircombo].append(new_qet)

#     # Inside of a two-symbol braket one may do three exchanges
#     # that must result in the same value. That if the wave
#     # functions are assumed to be real-valued.
#     if verbose:
#         msg = group_label + " Creating reality identities ..."
#         print(msg)
#     real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
#     # this has to run over all the quadruples of irs
#     kprimes = set()
#     for ir1, ir2 in product(*([group.irrep_labels]*2)):
#         real_var_simplifier = {}
#         ircombo = (ir1, ir2)
#         components = [component_labels[ir] for ir in ircombo]
#         for γ1, γ2 in product(*components):
#             k = (γ1, γ2)
#             kalt = (γ2, γ1)
#             # If for a given key I find that its
#             # switched version has already been seen
#             # Then that key has to be mapped to be
#             # mapped to the key already present.
#             if kalt in kprimes:
#                 real_var_simplifier[k] = kalt
#             else:
#                 real_var_simplifier[k] = k
#                 kprimes.add(k)
#         real_var_simplifiers_2[ircombo] = real_var_simplifier

#     # For each 2-tuple of irreps
#     # create a system of symbolic solutions
#     # and let sympy solve that.
#     # For each 2-tuple of irreps
#     # the end result is a dictionary
#     # whose keys represent the dependent
#     # brakets and whose values are the
#     # relation that those dependent values
#     # have with the independent brakets.
#     # As such, when these dictionaries are
#     # used on an expression, everything should
#     # then be given in terms of indepedent brakets.
#     if verbose:
#         msg = group_label + " Solving for independent 2-symbol brakets ..."
#         print(msg)
#     all_sols_2 = {irc:[] for irc in great_identities_2}
    
#     for ircombo in great_identities_2:
#         zeros = all_zeros_2[ircombo]
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols[ircombo] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
#         sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         sol_dict.update(zero_addendum)
#         all_sols_2[ircombo] = sol_dict

#     # This final dictionary is unnecessary but
#     # simplifies calling the replacements onto
#     # a symbolic expression.
#     if verbose:
#         msg = group_label + " Creating a dictionary with all the 2-symbol replacements ..."
#         print(msg)
    
#     super_solution_2 = {}
#     for ircombo in all_sols_2:
#         super_solution_2.update(all_sols_2[ircombo])

#     return real_var_full_simplifier, super_solution, real_var_simplifiers_2, super_solution_2

#     # for a given electron config
#     more_ids = {e_config:[] for e_config in simple_config_matrices}
#     for e_config in simple_config_matrices:
#         for term in simple_config_matrices[e_config]:
#             the_matrix = simple_config_matrices[e_config][term]
#             num_rows = len(the_matrix)
#             num_cols = len(the_matrix)
#             the_state_keys = config_supplement[e_config][term]
#             the_key_matrix = {}
#             for row_idx in range(num_rows):
#                 Γ3_row = the_state_keys[row_idx][2]
#                 γ3_row = the_state_keys[row_idx][3]
#                 for col_idx in range(num_cols):
#                     Γ3_col = the_state_keys[col_idx][2]
#                     γ3_col = the_state_keys[col_idx][3]
#                     matrix_val = the_matrix[row_idx][col_idx]
#                     the_key_matrix[(γ3_row,γ3_col)] = matrix_val
#             # now go over the keys of super_solution_2
#             # and if one of those keys matches with a key in the_key_matrix
#             # do something about it
#             for k in super_solution_2:
#                 if k in the_key_matrix:
#                     v = super_solution_2[k]
#                     # this matrix element
#                     matrix_element = the_key_matrix[k]
#                     # must be identified with the sum
#                     # as given in v
#                     matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
#                     identity = (matrix_element - matrix_equiv) #=0
#                     if len(identity.dict) != 0:
#                         more_ids[e_config].append(identity)
#                         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
#     if verbose:
#         msg = group_label + " Simplifying numeric values of qets ..."
#         pbar.set_description(msg)
#         pbar.refresh()
#         pbar.n += 1
    
#     more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}
    
#     if verbose:
#         msg = group_label + " Solving for dependent vars in terms of independent ones ..."
#         pbar.set_description(msg)
#         pbar.refresh()
#         pbar.n += 1
    
#     all_sols_2_4 = {irc:[] for irc in more_ids}
    
#     for e_config in more_ids:
#         problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
#         big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
#         big_mat = sp.Matrix(big_ma)
#         big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
#         rref_mat, pivots = big_mat.rref()
#         num_rows = rref_mat.rows
#         num_cols = rref_mat.cols
#         rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
#         rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
#         varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
#         eqns = rref_mat_non_zero*varvec
#         ssol = sp.solve(list(eqns), dict=True)
#         all_sols_2_4[e_config] = ssol
#         assert len(ssol) in [0,1]
#         if len(ssol) == 0:
#             sol_dict = {}
#         else:
#             sol_dict = ssol[0]
#         sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
#         all_sols_2_4[e_config] = sol_dict

#     # flatten into a single dictionary of replacements
#     fab_solution_2_4 = {}
#     for e_config in all_sols_2_4:
#         fab_solution_2_4.update(all_sols_2_4[e_config])
#     if verbose:
#         msg = group_label + " There are %d less independent variables ..."
#         pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
#         pbar.refresh()
#         pbar.n += 1
    
#     def simplifier_f(qet):
#         true_qet = Qet({})
#         for k,v in qet.dict.items():
#             if k in fab_solution_2_4:
#                 true_qet += v*Qet(fab_solution_2_4[k])
#             else:
#                 true_qet += Qet({k:v})
#         return true_qet
    
#     def simplify_config_matrix_f(ir1ir2, S, ir3):
#         config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
#         num_rows = len(config_matrix)
#         simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
#         #simple_matrix = sp.Matrix(simple_matrix)
#         return simple_matrix
    
#     if verbose:
#         msg = group_label + " Making final simplifications ..."
#         pbar.set_description(msg)
#         pbar.refresh()
#         pbar.n += 1
    
#     final_config_matrices = {k:{} for k in config_matrices}
#     for ir1ir2 in config_matrices.keys():
#         for term_key in config_matrices[ir1ir2]:
#             S, Γ3 = term_key
#             final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)
    
#     if verbose:
#         msg = group_label + " Finished"
#         pbar.set_description(msg)
#         pbar.refresh()
    
#     return final_config_matrices

O Creating all 4-symbol identities ...
O Creating set of identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 4-symbol brakets ...
O Creating a dictionary with all the 4-symbol replacements ...
O Creating all 2-symbol identities ...
O Creating set of 2 symbol identities ...
O Finding trivial zeros ...
O Using them to simplify things ...
O Creating reality identities ...
O Solving for independent 2-symbol brakets ...
O Creating a dictionary with all the 2-symbol replacements ...


In [38]:
four_real_var_ids

{({\alpha}, {\alpha}, {\alpha}, {\alpha}): ({\alpha},
  {\alpha},
  {\alpha},
  {\alpha}),
 ({\alpha}, {\alpha}, {\alpha}, {\beta}): ({\alpha},
  {\alpha},
  {\alpha},
  {\beta}),
 ({\alpha}, {\alpha}, {\alpha}, {\gamma}): ({\alpha},
  {\alpha},
  {\alpha},
  {\gamma}),
 ({\alpha}, {\alpha}, {\alpha}, {\zeta}): ({\alpha},
  {\alpha},
  {\alpha},
  {\zeta}),
 ({\alpha}, {\alpha}, {\alpha}, {\eta}): ({\alpha},
  {\alpha},
  {\alpha},
  {\eta}),
 ({\alpha}, {\alpha}, {\alpha}, {\mu}): ({\alpha}, {\alpha}, {\alpha}, {\mu}),
 ({\alpha}, {\alpha}, {\alpha}, {\nu}): ({\alpha}, {\alpha}, {\alpha}, {\nu}),
 ({\alpha}, {\alpha}, {\alpha}, {\xi}): ({\alpha}, {\alpha}, {\alpha}, {\xi}),
 ({\alpha}, {\alpha}, {\alpha}, {\phi}): ({\alpha},
  {\alpha},
  {\alpha},
  {\phi}),
 ({\alpha}, {\alpha}, {\alpha}, {\chi}): ({\alpha},
  {\alpha},
  {\alpha},
  {\chi}),
 ({\alpha}, {\alpha}, {\beta}, {\alpha}): ({\alpha},
  {\alpha},
  {\beta},
  {\alpha}),
 ({\alpha}, {\alpha}, {\beta}, {\beta}): ({\alpha},
 

In [35]:
for k in set(sum(four_zeros.values(),[])):
    print(four_ids[k])

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


{({\alpha}, {\alpha}, {\alpha}, {\beta}),
 ({\alpha}, {\alpha}, {\alpha}, {\mu}),
 ({\alpha}, {\alpha}, {\alpha}, {\phi}),
 ({\alpha}, {\alpha}, {\alpha}, {\zeta}),
 ({\alpha}, {\alpha}, {\beta}, {\alpha}),
 ({\alpha}, {\alpha}, {\beta}, {\gamma}),
 ({\alpha}, {\alpha}, {\beta}, {\nu}),
 ({\alpha}, {\alpha}, {\beta}, {\xi}),
 ({\alpha}, {\alpha}, {\gamma}, {\beta}),
 ({\alpha}, {\alpha}, {\gamma}, {\mu}),
 ({\alpha}, {\alpha}, {\gamma}, {\zeta}),
 ({\alpha}, {\alpha}, {\mu}, {\alpha}),
 ({\alpha}, {\alpha}, {\mu}, {\gamma}),
 ({\alpha}, {\alpha}, {\mu}, {\xi}),
 ({\alpha}, {\alpha}, {\nu}, {\beta}),
 ({\alpha}, {\alpha}, {\nu}, {\phi}),
 ({\alpha}, {\alpha}, {\phi}, {\alpha}),
 ({\alpha}, {\alpha}, {\phi}, {\nu}),
 ({\alpha}, {\alpha}, {\xi}, {\beta}),
 ({\alpha}, {\alpha}, {\xi}, {\mu}),
 ({\alpha}, {\alpha}, {\xi}, {\zeta}),
 ({\alpha}, {\alpha}, {\zeta}, {\alpha}),
 ({\alpha}, {\alpha}, {\zeta}, {\gamma}),
 ({\alpha}, {\alpha}, {\zeta}, {\xi}),
 ({\alpha}, {\beta}, {\alpha}, {\gamma

## Two electron operators

In [306]:
t2e2 = CrystalElectronsLLcoupling('O',[sp.Symbol('T_2'),2],[sp.Symbol('E'),2])

In [307]:
# def double_electron_braket(qet0, qet1, op):
#     full_braket = []
#     for det0, coeff0 in qet0.dict.items():
#         num_electrons = len(det0)
#         for det1, coeff1 in qet1.dict.items():
#             # before given value to the braket it is necessary to align the symbols in the determinantal states
#             # and keep track of the reordering sign
#             common_symbs = list(set(det0).intersection(set(det1)))
#             different_symbs0 = [x for x in det0 if x not in common_symbs]
#             different_symbs1 = [x for x in det1 if x not in common_symbs]
#             # there are no repeat symbols in any determinantal state
#             newdet0 = different_symbs0 + common_symbs
#             newdet1 = different_symbs1 + common_symbs
#             ordering0 = [det0.index(x) for x in newdet0]
#             ordering1 = [det1.index(x) for x in newdet1]
#             extra_sign = εijk(*ordering0) * εijk(*ordering1)
#             total_coeff = extra_sign * coeff0 * coeff1
#             odet0 = newdet0
#             odet1 = newdet1
#             double_brakets = []
#             if odet0 == odet1:
#                 # CASE I
#                 for i in range(num_electrons):
#                     for j in range(i-1,num_electrons):
#                         spin_up_0_i = 'bar' not in str(odet0[i])
#                         spin_up_0_j = 'bar' not in str(odet0[j])
#                         double_brakets.append(((odet0[i], odet0[j],
#                                                 op,
#                                                 odet0[i], odet0[j]),
#                                                 total_coeff))
#                         if (spin_up_0_i == spin_up_0_j): 
#                             double_brakets.append(((odet0[i], odet0[j],
#                                                     op,
#                                                     odet0[j], odet0[i]),
#                                                     -total_coeff))
#             elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
#                 # CASE II
#                 spin_up_0_0 = ('bar' not in str(odet0[0]))
#                 spin_up_1_0 = ('bar' not in str(odet1[0]))
#                 for j in range(1,num_electrons):
#                     spin_up_0_j = ('bar' not in str(odet0[j]))
#                     if (spin_up_0_0 == spin_up_1_0):
#                         double_brakets.append(((odet0[0], odet0[j],
#                                                 op,
#                                                 odet1[0], odet0[j]),
#                                                 total_coeff))
#                     if (spin_up_0_0 == spin_up_0_j) and (spin_up_0_j == spin_up_1_0): 
#                         double_brakets.append(((odet0[0], odet0[j],
#                                                 op,
#                                                 odet0[j], odet1[0]),
#                                                 -total_coeff))
#             elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]) and all([(odet0[i] == odet1[i]) for i in range(2,num_electrons)]):
#                 # CASE III
#                 print(3)
#                 spin_up_0_0 = ('bar' not in str(odet0[0]))
#                 spin_up_0_1 = ('bar' not in str(odet0[1]))
#                 spin_up_1_0 = ('bar' not in str(odet1[0]))
#                 spin_up_1_1 = ('bar' not in str(odet1[1]))
#                 if (spin_up_0_0 == spin_up_1_0) and (spin_up_0_1 == spin_up_1_1):
#                     double_brakets.append[((odet0[0], odet0[1],
#                                             op,
#                                             odet1[0], odet1[1]),
#                                             total_coeff)]
#                 if (spin_up_0_0 == spin_up_1_1) and (spin_up_0_1 == spin_up_1_0):
#                     double_brakets.append[((odet0[0], odet0[1],
#                                             op,
#                                             odet1[1], odet1[0]),
#                                             -total_coeff)]
#             else:
#                 # CASE III
#                 pass
#             full_braket.extend(double_brakets)
#     full_braket = Qet(dict(full_braket))
#     return full_braket

In [308]:
qet0 = Qet({(sp.Symbol(r'\bar{v}'), sp.Symbol(r'\eta'), sp.Symbol(r'\bar{\zeta}')):1})
qet1 = Qet({(sp.Symbol(r'\bar{u}'), sp.Symbol(r'\eta'), sp.Symbol(r'\bar{\zeta}')):1})
as_braket_with_operator(double_electron_braket(qet0, qet1, sp.Symbol(r'G')))

-\langle{\bar{v}\bar{\zeta}}|\hat{G}|{\bar{\zeta}\bar{u}}\rangle + \langle{\bar{v}\bar{\zeta}}|\hat{G}|{\bar{u}\bar{\zeta}}\rangle + \langle{\bar{v}\eta}|\hat{G}|{\bar{u}\eta}\rangle

In [195]:
many_brakets = []
for qet0 in list(t2e2.equiv_waves.values()):
    for qet1 in list(t2e2.equiv_waves.values()):
        many_brakets.append(as_double_electron_braket(double_electron_braket(qet0,qet1,sp.Symbol('f_12'))))
    break
        

In [196]:
for b in many_brakets:
    if b != 0:
        display(b)
        break

-\langle\bar{{\chi}}\bar{{\chi}}|f_{12}|\bar{{\chi}}\bar{{\chi}}\rangle/6 - \langle\bar{{\chi}}\bar{{\gamma}}|f_{12}|\bar{{\gamma}}\bar{{\chi}}\rangle/6 - \langle\bar{{\chi}}\bar{{\zeta}}|f_{12}|\bar{{\zeta}}\bar{{\chi}}\rangle/6 - \langle\bar{{\gamma}}\bar{{\chi}}|f_{12}|\bar{{\chi}}\bar{{\gamma}}\rangle/6 - \langle\bar{{\gamma}}\bar{{\gamma}}|f_{12}|\bar{{\gamma}}\bar{{\gamma}}\rangle/6 - \langle\bar{{\gamma}}\bar{{\phi}}|f_{12}|\bar{{\phi}}\bar{{\gamma}}\rangle/6 - \langle\bar{{\gamma}}\bar{{\xi}}|f_{12}|\bar{{\xi}}\bar{{\gamma}}\rangle/6 - \langle\bar{{\phi}}\bar{{\phi}}|f_{12}|\bar{{\phi}}\bar{{\phi}}\rangle/6 - \langle\bar{{\xi}}\bar{{\gamma}}|f_{12}|\bar{{\gamma}}\bar{{\xi}}\rangle/6 - \langle\bar{{\xi}}\bar{{\xi}}|f_{12}|\bar{{\xi}}\bar{{\xi}}\rangle/6 - \langle\bar{{\xi}}\bar{{\zeta}}|f_{12}|\bar{{\zeta}}\bar{{\xi}}\rangle/6 - \langle\bar{{\zeta}}\bar{{\chi}}|f_{12}|\bar{{\chi}}\bar{{\zeta}}\rangle/6 - \langle\bar{{\zeta}}\bar{{\phi}}|f_{12}|\bar{{\phi}}\bar{{\zeta}}\rangle/6 

In [202]:
newdet0

[\eta, \bar{\zeta}, \bar{v}]

In [208]:
qet0 = Qet({(sp.Symbol(r'\bar{v}'), sp.Symbol(r'\eta'), sp.Symbol(r'\bar{\zeta}')):1})
qet1 = Qet({(sp.Symbol(r'\bar{u}'), sp.Symbol(r'\eta'), sp.Symbol(r'\bar{\zeta}')):1})
full_braket = []
for det0, coeff0 in qet0.dict.items():
    for det1, coeff1 in qet1.dict.items():
        num_electrons = len(det0)
        # before given value to the braket it is necessary to align the symbols in the determinantal states
        # and keep track of the reordering sign
        common_symbs = list(set(det0).intersection(set(det1)))
        different_symbs0 = [x for x in det0 if x not in common_symbs]
        different_symbs1 = [x for x in det1 if x not in common_symbs]
        # there are no repeat symbols in any determinantal state
        newdet0 = different_symbs0 + common_symbs
        newdet1 = different_symbs1 + common_symbs
        ordering0 = [det0.index(x) for x in newdet0]
        ordering1 = [det1.index(x) for x in newdet1]
        extra_sign = εijk(*ordering0) * εijk(*ordering1)
        coeff = extra_sign*coeff0*coeff1
        odet0 = newdet0
        odet1 = newdet1
        double_brakets = []
        if odet0 == odet1:
            print(1)
            for i in range(num_electrons):
                for j in range(i-1,num_electrons):
                    spin_up_i = 'bar' not in str(odet0[i])
                    spin_up_j = 'bar' not in str(odet0[j])
                    if (spin_up_i == spin_up_j): 
                        double_brakets.append(((odet0[i], odet0[j],
                                                op,
                                                odet0[j], odet0[i]),
                                                -coeff))
        elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
            print(2)
            spin_up_0 = 'bar' not in str(odet0[0])
            for i in range(1,num_electrons):
                spin_up_i = 'bar' not in str(odet0[i])
                double_brakets.append[((odet0[0],odet0[i],op,odet0[0],odet0[i]),coeff)]
                if (spin_up_1 == spin_up_i): 
                    double_brakets.append[((odet0[0],odet0[i],op,odet0[i],odet0[0]),-coeff)]
        elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]) and (odet0[2] != odet1[2]):
            print(3)
            double_brakets.append[((odet0[1],odet0[2],op,odet0[1],odet0[2]),coeff)]
            double_brakets.append[((odet0[1],odet0[2],op,odet0[2],odet0[1]),-coeff)]
        else:
            print(4)
        full_braket.extend(double_brakets)
print(full_braket)

2


NameError: name 'op' is not defined

In [206]:
odet1

[\eta, \bar{\zeta}, \bar{u}]

In [207]:
odet0

[\eta, \bar{\zeta}, \bar{v}]

In [198]:
qet0.as_ket()

|(\bar{v}, \eta, \bar{\zeta})>

In [ ]:
# the more complex braket, i.e. something between arbitrary qets, is built
# from the simpler brakets for "pure" determinantal states
qet0 = list(t2e2.equiv_waves.values())[0]
qet1 = list(t2e2.equiv_waves.values())[1]
full_braket = []
for idx0 in range(len(qet0.dict.keys())):
    for idx1 in range(len(qet1.dict.keys())):
        det0 = list(qet0.dict.keys())[0]
        det1 = list(qet1.dict.keys())[1]
        num_electrons = len(det0)
        f2 = sp.Symbol('f_1,2')
        # before given value to the braket it is necessary to align the symbols in the determinantal states
        # and keep track of the reordering sign
        common_symbs = list(set(det0).intersection(set(det1)))
        different_symbs0 = [x for x in det0 if x not in common_symbs]
        different_symbs1 = [x for x in det1 if x not in common_symbs]
        # there are no repeat symbols in any determinantal state
        newdet0 = common_symbs + different_symbs0
        newdet1 = common_symbs + different_symbs1
        ordering0 = [det0.index(x) for x in newdet0]
        ordering1 = [det1.index(x) for x in newdet1]
        extra_sign = εijk(*ordering0) * εijk(*ordering1)
        odet0 = newdet0
        odet1 = newdet1
        if odet0 == odet1:
            double_brakets = []
            for i in range(num_electrons):
                for j in range(i-1,num_electrons):
                    double_brakets.append(((odet0[i],odet0[j],f2,odet0[i],odet0[j]),extra_sign))
                    double_brakets.append(((odet0[i],odet0[j],f2,odet0[j],odet0[i]),-extra_sign))
        elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
            double_brakets = []
            for i in range(2,num_electrons):
                double_brakets.append[((odet0[1],odet0[j],f2,odet0[1],odet0[j]),extra_sign)]
                double_brakets.append[((odet0[1],odet0[j],f2,odet0[j],odet0[1]),-extra_sign)]
        elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]) and all([(odet0[i] == odet1[i]) for i in range(2,num_electrons)]):
            double_brakets.append[((odet0[1],odet0[2],f2,odet0[1],odet0[2]),extra_sign)]
            double_brakets.append[((odet0[1],odet0[2],f2,odet0[2],odet0[1]),-extra_sign)]
        else:
            double_brakets = []
        full_braket.extend(double_brakets)
double_brakets = Qet(dict(double_brakets))
as_double_electron_braket(double_brakets)

In [177]:
# the more complex braket, i.e. something between arbitrary qets, is built
# from the simpler brakets for "pure" determinantal states
qet0 = list(t2e2.equiv_waves.values())[0]
qet1 = list(t2e2.equiv_waves.values())[1]
full_braket = []
for idx0 in range(len(qet0.dict.keys())):
    for idx1 in range(len(qet1.dict.keys())):
        det0 = list(qet0.dict.keys())[0]
        det1 = list(qet1.dict.keys())[1]
        num_electrons = len(det0)
        f2 = sp.Symbol('f_1,2')
        # before given value to the braket it is necessary to align the symbols in the determinantal states
        # and keep track of the reordering sign
        common_symbs = list(set(det0).intersection(set(det1)))
        different_symbs0 = [x for x in det0 if x not in common_symbs]
        different_symbs1 = [x for x in det1 if x not in common_symbs]
        # there are no repeat symbols in any determinantal state
        newdet0 = common_symbs + different_symbs0
        newdet1 = common_symbs + different_symbs1
        ordering0 = [det0.index(x) for x in newdet0]
        ordering1 = [det1.index(x) for x in newdet1]
        extra_sign = εijk(*ordering0) * εijk(*ordering1)
        odet0 = newdet0
        odet1 = newdet1
        if odet0 == odet1:
            double_brakets = []
            for i in range(num_electrons):
                for j in range(i-1,num_electrons):
                    double_brakets.append(((odet0[i],odet0[j],f2,odet0[i],odet0[j]),extra_sign))
                    double_brakets.append(((odet0[i],odet0[j],f2,odet0[j],odet0[i]),-extra_sign))
        elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
            double_brakets = []
            for i in range(2,num_electrons):
                double_brakets.append[((odet0[1],odet0[j],f2,odet0[1],odet0[j]),extra_sign)]
                double_brakets.append[((odet0[1],odet0[j],f2,odet0[j],odet0[1]),-extra_sign)]
        elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]) and all([(odet0[i] == odet1[i]) for i in range(2,num_electrons)]):
            double_brakets.append[((odet0[1],odet0[2],f2,odet0[1],odet0[2]),extra_sign)]
            double_brakets.append[((odet0[1],odet0[2],f2,odet0[2],odet0[1]),-extra_sign)]
        else:
            double_brakets = []
        full_braket.extend(double_brakets)
double_brakets = Qet(dict(double_brakets))
as_double_electron_braket(double_brakets)

0

In [170]:
# the more complex braket, i.e. something between arbitrary qets, is built
# from the simpler brakets for "pure" determinantal states
qet0 = list(t2e2.equiv_waves.values())[0]
qet1 = list(t2e2.equiv_waves.values())[1]
det0 = list(qet0.dict.keys())[0]
det1 = list(qet1.dict.keys())[1]
num_electrons = len(det0)
f2 = sp.Symbol('f_1,2')
# before given value to the braket it is necessary to align the symbols in the determinantal states
# and keep track of the reordering sign
common_symbs = list(set(det0).intersection(set(det1)))
different_symbs0 = [x for x in det0 if x not in common_symbs]
different_symbs1 = [x for x in det1 if x not in common_symbs]
# there are no repeat symbols in any determinantal state
newdet0 = common_symbs + different_symbs0
newdet1 = common_symbs + different_symbs1
ordering0 = [det0.index(x) for x in newdet0]
ordering1 = [det1.index(x) for x in newdet1]
extra_sign = εijk(*ordering0) * εijk(*ordering1)
odet0 = newdet0
odet1 = newdet1
if odet0 == odet1:
    double_brakets = []
    for i in range(num_electrons):
        for j in range(i-1,num_electrons):
            double_brakets.append(((odet0[i],odet0[j],f2,odet0[i],odet0[j]),extra_sign))
            double_brakets.append(((odet0[i],odet0[j],f2,odet0[j],odet0[i]),-extra_sign))
elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
    double_brakets = []
    for i in range(2,num_electrons):
        double_brakets.append[((odet0[1],odet0[j],f2,odet0[1],odet0[j]),extra_sign)]
        double_brakets.append[((odet0[1],odet0[j],f2,odet0[j],odet0[1]),-extra_sign)]
elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]) and all([(odet0[i] == odet1[i]) for i in range(2,num_electrons)]):
    double_brakets.append[((odet0[1],odet0[2],f2,odet0[1],odet0[2]),extra_sign)]
    double_brakets.append[((odet0[1],odet0[2],f2,odet0[2],odet0[1]),-extra_sign)]
else:
    double_brakets = []
double_brakets = Qet(dict(double_brakets))
as_double_electron_braket(double_brakets)

0

In [168]:
num_electrons = 4
for i in range(num_electrons):
    for j in range(i+1,num_electrons):
        print(i,j)

0 1
0 2
0 3
1 2
1 3
2 3


In [163]:
double_brakets

Qet({({\xi}, {\xi}, f_1,2, {\xi}, {\xi}): 1, ({\xi}, \bar{{\xi}}, f_1,2, {\xi}, \bar{{\xi}}): 1, ({\xi}, \bar{{\gamma}}, f_1,2, {\xi}, \bar{{\gamma}}): 1, ({\xi}, {\gamma}, f_1,2, {\xi}, {\gamma}): 1, (\bar{{\xi}}, \bar{{\xi}}, f_1,2, \bar{{\xi}}, \bar{{\xi}}): 1, (\bar{{\xi}}, \bar{{\gamma}}, f_1,2, \bar{{\xi}}, \bar{{\gamma}}): 1, (\bar{{\xi}}, {\gamma}, f_1,2, \bar{{\xi}}, {\gamma}): 1, (\bar{{\gamma}}, \bar{{\gamma}}, f_1,2, \bar{{\gamma}}, \bar{{\gamma}}): 1, (\bar{{\gamma}}, {\gamma}, f_1,2, \bar{{\gamma}}, {\gamma}): 1, ({\gamma}, {\gamma}, f_1,2, {\gamma}, {\gamma}): 1})

In [157]:
num_electrons
[tuple(1 for i in range(j,num_electrons)) for j in range(num_electrons)]

[(1, 1, 1, 1), (1, 1, 1), (1, 1), (1,)]

## One Electron Operators

How to include operators into qdef?

Partly what I want is a function that takes two qets, and returns a qet of brakets (with single electron bras and kets) with an assumed single-electron operator in between.
Basically I need to discriminate between cases I, II, and III in STK 3.33, 3.34, and 3.35.

For testing I shall use the qets for t_2^2 * e^2.

In [71]:
t2e2 = CrystalElectronsLLcoupling('O',[sp.Symbol('T_2'),2],[sp.Symbol('E'),2])

In [91]:
# the more complex braket, i.e. something between arbitrary qets, is built
# from the simpler brakets for "pure" determinantal states
qet0 = list(t2e2.equiv_waves.values())[0]
qet1 = list(t2e2.equiv_waves.values())[1]
det0 = list(qet0.dict.keys())[0]
det1 = list(qet1.dict.keys())[1]
num_electrons = len(det0)
f1 = sp.Symbol('f_1')
# before given value to the braket it is necessary to align the symbols in the determinantal states
# and keep track of the reordering sign
common_symbs = list(set(det0).intersection(set(det1)))
different_symbs0 = [x for x in det0 if x not in common_symbs]
different_symbs1 = [x for x in det1 if x not in common_symbs]
# there are no repeat symbols in any determinantal state
newdet0 = common_symbs + different_symbs0
newdet1 = common_symbs + different_symbs1
ordering0 = [det0.index(x) for x in newdet0]
ordering1 = [det1.index(x) for x in newdet1]
extra_sign = εijk(*ordering0) * εijk(*ordering1)
odet0 = newdet0
odet1 = newdet1
if odet0 == odet1:
    single_brakets = [((φ,f1,φ),extra_sign) for φ in odet0]
elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
    single_brakets = [((odet0[0],f1,odet1[0]),extra_sign)]
elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]):
    single_brakets = []
else:
    single_brakets = []
single_brakets = Qet(dict(single_brakets))

In [114]:
def as_singe_electron_braket(qet):
    tot = 0
    for k,v in qet.dict.items():
        p = v*sp.Symbol(r'\langle%s|%s|%s\rangle' % (sp.latex(k[0]), sp.latex(k[1]), sp.latex(k[2])))
        tot += p
    return tot

In [119]:
# the more complex braket, i.e. something between arbitrary qets, is built
# from the simpler brakets for "pure" determinantal states
qet0 = list(t2e2.equiv_waves.values())[0]
qet1 = list(t2e2.equiv_waves.values())[1]
full_braket = []
for idx0 in range(len(qet0.dict.keys())):
    for idx1 in range(len(qet1.dict.keys())):
        det0 = list(qet0.dict.keys())[idx0]
        det1 = list(qet1.dict.keys())[idx1]
        num_electrons = len(det0)
        f1 = sp.Symbol('f_1')
        # before given value to the braket it is necessary to align the symbols in the determinantal states
        # and keep track of the reordering sign
        common_symbs = list(set(det0).intersection(set(det1)))
        different_symbs0 = [x for x in det0 if x not in common_symbs]
        different_symbs1 = [x for x in det1 if x not in common_symbs]
        # there are no repeat symbols in any determinantal state
        newdet0 = different_symbs0 + common_symbs
        newdet1 = different_symbs1 + common_symbs
        ordering0 = [det0.index(x) for x in newdet0]
        ordering1 = [det1.index(x) for x in newdet1]
        extra_sign = εijk(*ordering0) * εijk(*ordering1)
        odet0 = newdet0
        odet1 = newdet1
        if odet0 == odet1:
            single_brakets = [((φ,f1,φ),extra_sign) for φ in odet0]
        elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
            single_brakets = [((odet0[0],f1,odet1[0]),extra_sign)]
        elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]):
            single_brakets = []
        if len(single_brakets) != 0:
            # single_brakets = Qet(dict(single_brakets))
            full_braket.extend(single_brakets)
full_braket = Qet(dict(full_braket))
as_singe_electron_braket(full_braket)

\langle\bar{{\chi}}|f_{1}|\bar{{\chi}}\rangle + \langle\bar{{\gamma}}|f_{1}|\bar{{\gamma}}\rangle + \langle\bar{{\phi}}|f_{1}|\bar{{\phi}}\rangle + \langle\bar{{\xi}}|f_{1}|\bar{{\xi}}\rangle + \langle\bar{{\zeta}}|f_{1}|\bar{{\zeta}}\rangle + \langle{\chi}|f_{1}|{\chi}\rangle + \langle{\gamma}|f_{1}|{\gamma}\rangle + \langle{\phi}|f_{1}|{\phi}\rangle + \langle{\xi}|f_{1}|{\xi}\rangle + \langle{\zeta}|f_{1}|{\zeta}\rangle

In [141]:
# def single_electron_braket(qet0, qet1, op):
#     '''
#     Given two qets, assumed to be composed of determinantal states, and an operator, assumed to 
#     be a single-electron operator.
#     Return value of the braket <qet0| \sum_1^num_electrons f_i |qet1>

#     Parameters
#     ----------
#     qet0    (qdefcore.Qet)
#     qet1    (qdefcore.Qet)
#     op      (sp.Symbol)

#     Returns
#     -------
#     braket  (qdefcore.Qet): with each key having three symbols, first one equal to a single electron
#                             orbital, second one equal to the provided single electron operator, and
#                             the third one equal to another single electron orbital. Interpreted as
#                             <φi | f | φj>
    
#     '''
#     full_braket = []
#     for det0, coeff0 in qet0.dict.items():
#         for det1, coeff1 in qet1.dict.items():
#             num_electrons = len(det0)
#             # before given value to the braket it is necessary to align the symbols in the determinantal states
#             # and keep track of the reordering sign
#             set0 = set(det0)
#             set1 = set(det1)
#             # there should be no repeat symbols in any determinantal state
#             assert len(set0) == len(det0) and len(set1) == len(det1), "There's something funny here..."
#             common_symbs = list(set0.intersection(set1))
#             different_symbs0 = [x for x in det0 if x not in common_symbs]
#             different_symbs1 = [x for x in det1 if x not in common_symbs]
#             newdet0 = different_symbs0 + common_symbs
#             newdet1 = different_symbs1 + common_symbs
#             ordering0 = [det0.index(x) for x in newdet0]
#             ordering1 = [det1.index(x) for x in newdet1]
#             extra_sign = εijk(*ordering0) * εijk(*ordering1)
#             total_coeff = coeff0 * coeff1 * extra_sign
#             odet0 = newdet0
#             odet1 = newdet1
#             if odet0 == odet1:
#                 single_brakets = [((φ,op,φ),total_coeff) for φ in odet0]
#             elif (odet0[0] != odet1[0]) and all([(odet0[i] == odet1[i]) for i in range(1,num_electrons)]):
#                 single_brakets = [((odet0[0],op,odet1[0]),total_coeff)]
#             elif (odet0[0] != odet1[0]) and (odet0[1] != odet1[1]):
#                 single_brakets = []
#             if len(single_brakets) != 0:
#                 full_braket.extend(single_brakets)
#     full_braket = Qet(dict(full_braket))
#     return full_braket

In [142]:
many_brakets = []
for qet0 in list(t2e2.equiv_waves.values()):
    for qet1 in list(t2e2.equiv_waves.values()):
        many_brakets.append(as_singe_electron_braket(single_electron_braket(qet0,qet1,sp.Symbol('f'))))
        

In [147]:
counter = 0
for q in many_brakets:
    if q !=0:
        display(q)
        counter += 1
        if counter == 3:
            break

\langle\bar{{\chi}}|f|\bar{{\chi}}\rangle/6 + \langle\bar{{\gamma}}|f|\bar{{\gamma}}\rangle/6 + \langle\bar{{\phi}}|f|\bar{{\phi}}\rangle/6 + \langle\bar{{\xi}}|f|\bar{{\xi}}\rangle/6 + \langle\bar{{\zeta}}|f|\bar{{\zeta}}\rangle/6 + \langle{\chi}|f|{\chi}\rangle/6 + \langle{\gamma}|f|{\gamma}\rangle/6 + \langle{\phi}|f|{\phi}\rangle/6 + \langle{\xi}|f|{\xi}\rangle/6 + \langle{\zeta}|f|{\zeta}\rangle/6

\langle\bar{{\chi}}|f|\bar{{\chi}}\rangle/6 - \langle\bar{{\gamma}}|f|\bar{{\gamma}}\rangle/6 + \langle\bar{{\phi}}|f|\bar{{\phi}}\rangle/6 + \langle\bar{{\xi}}|f|\bar{{\xi}}\rangle/6 + \langle\bar{{\zeta}}|f|\bar{{\zeta}}\rangle/6 + \langle{\chi}|f|{\chi}\rangle/6 - \langle{\gamma}|f|{\gamma}\rangle/6 + \langle{\phi}|f|{\phi}\rangle/6 + \langle{\xi}|f|{\xi}\rangle/6 + \langle{\zeta}|f|{\zeta}\rangle/6

sqrt(2)*\langle{\gamma}|f|\bar{{\zeta}}\rangle/6 - sqrt(2)*\langle{\zeta}|f|\bar{{\gamma}}\rangle/6